In [3]:
# Install the OpenAI and LangChain libraries
# - `openai`: Provides access to OpenAI's GPT models for tasks like text generation, embeddings, and completions.
# - `langchain`: A framework for building applications using large language models (LLMs).
#                Includes tools for chaining prompts, memory, and integrations like knowledge graphs.
!pip install -q openai langchain
# Attempt to install the LangChain Community library
# - `langchain-community`: This may refer to a community-supported version or extensions of LangChain.
#   Ensure this package exists and is maintained if errors occur during installation.
!pip install -q langchain-community

This script initializes the OpenAI API client and defines a function to interact with the GPT model. The get_chat_response function sends a user-provided text input to the GPT model (gpt-3.5-turbo) and returns the model's response.

In [4]:

import os
from openai import OpenAI

# Set the API key in the environment variable
os.environ["OPENAI_API_KEY"] = "sk-MNL1gYbV6CyXkh2rwPxao_D7n8nSxwW4_0wozr5sUtT3BlbkFJoEpwVXUH_Z3deg71NI-mM8QqSOkOGzQ5WDXmQ8FQEA" # Replace with your actual API key

client = OpenAI()

def get_chat_response(text):
    """
    This function takes a text input and returns the chat completion message.
    """
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": text,
            }
        ],
        model="gpt-3.5-turbo",
    )
    return chat_completion.choices[0].message.content


In [ ]:
import networkx as nx  # For creating and analyzing graphs/networks.

import matplotlib.pyplot as plt  # For data visualization and plotting.

import numpy as np  # For numerical operations and array handling.

import seaborn as sns

import pandas as pd

import random  # For generating random numbers.

from langchain.llms import OpenAI

from langchain.graphs.networkx_graph import NetworkxEntityGraph, KnowledgeTriple # Represents (subject, predicate, object) triples.

from scipy.spatial.distance import cosine  # For cosine similarity/distance between vectors.

from scipy.stats import wasserstein_distance  # For Wasserstein distance (probability distribution comparison).

from sklearn.metrics.pairwise import cosine_similarity

from sklearn.linear_model import LinearRegression, BayesianRidge  # Regression models.

from sklearn.datasets import fetch_20newsgroups  # Fetch the 20 Newsgroups text dataset.

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score  # Model performance metrics.

import matplotlib.colors as mcolors  # For handling and customizing colors in visualizations.

import sklearn.metrics  # For evaluation metrics like accuracy, precision, recall, etc.

import matplotlib.colors as mcolors  # For handling color schemes in plots

import textwrap  # For wrapping text into fixed-width lines

from sklearn.metrics import roc_curve, auc

from langchain.chains import GraphQAChain  # For question answering over knowledge graphs.

# Prompt Engineering
from langchain.prompts import PromptTemplate  # To define templates for LLM prompts.


In [ ]:
!pip install  rdflib  SPARQLWrapper

This script defines a knowledge graph using a set of triples representing entities (nodes) and their relationships (edges). The triples are categorized into parts based on themes, such as LLMs in the legal context, RAG integration, collaborations, and key people involved. The knowledge graph is constructed programmatically by adding these triples into the graph index, which allows for efficient querying and analysis.

In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON

# Set up the DBpedia SPARQL endpoint
sparql = SPARQLWrapper("https://dbpedia.org/sparql")

# SPARQL Query: Retrieve cybersecurity-related concepts and their triples
query = """
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?concept ?label ?abstract
WHERE {
  ?concept rdf:type dbo:Software .
  ?concept rdfs:label ?label .
  ?concept dbo:abstract ?abstract .
  FILTER (LANG(?label) = 'en' && LANG(?abstract) = 'en')
  FILTER (CONTAINS(LCASE(?label), "cyber") || CONTAINS(LCASE(?label), "security") || CONTAINS(LCASE(?label), "malware"))
}
LIMIT 10
"""

# Execute the query
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

# Initialize Knowledge Graph (KG) and portion tracking
kg = []
portion_indices = {}
portion_counter = 1  # Start portion numbering
triple_index = 0  # Track overall index

print("\nStructured Knowledge Graph:\n")

for result in results["results"]["bindings"]:
    concept = result["concept"]["value"].split("/")[-1]  # Extracts entity name
    label = result["label"]["value"]
    abstract = result["abstract"]["value"]  # Store full abstract without truncation

    # Store portion index range
    start_index = triple_index
    portion_indices[f"Part {portion_counter}"] = range(start_index, start_index + 3)  # Each part has 3 triples

    # Print structured output
    print(f"\n# Part {portion_counter}")
    print(f"({concept}) → (type) → (Software)")
    print(f"({concept}) → (label) → ({label})")
    print(f"({concept}) → (abstract) →")
    print(abstract)  # Print full abstract with line breaks
    print("-" * 80)

    # Store in KG
    kg.append((concept, "type", "Software"))
    kg.append((concept, "label", label))
    kg.append((concept, "abstract", abstract))  # Store full abstract

    # Increment indices
    triple_index += 3
    portion_counter += 1

# Print portion indices separately
print("\nPortion Indices:\n")
for part, index_range in portion_indices.items():
    print(f"{part}: {index_range}")

# Save KG to a text file
with open("knowledge_graph_output.txt", "w", encoding="utf-8") as f:
    for triple in kg:
        f.write(f"( {triple[0]} , {triple[1]} , {triple[2]})\n\n")  # Ensuring full visibility

print("\nFinal Knowledge Graph saved as 'knowledge_graph_output.txt'.")

# Print the final KG in a readable format
print("\nFinal Knowledge Graph List:\n")
for triple in kg:
  print("(", triple[0],",", triple[1],", ",triple[2], ")")  # Print without truncation

In [ ]:
print("Original KG node count:", len(set(node for triple in kg for node in (triple[0], triple[2]))))

In [ ]:
part_indices ={
"Part 1": range(0, 3),
"Part 2": range(3, 6),
"Part 3": range(6, 9),
"Part 4": range(9, 12),
"Part 5": range(12, 15),
"Part 6": range(15, 18),
"Part 7": range(18, 21),
"Part 8": range(21, 24),
"Part 9": range(24, 27),
"Part 10": range(27, 30)
}
part_names = list(part_indices.keys())

# Instantiate the graph
graph = NetworkxEntityGraph()

# Build the graph from the knowledge triples
for (node1, relation, node2) in kg:
    graph.add_triple(KnowledgeTriple(node1, relation, node2))

In [ ]:
# Function to wrap text for better display
def wrap_text(text, max_words=8):
    """Wrap text if it contains more than `max_words` words."""
    words = text.split()
    return "Explanation" if len(words) > max_words else text

Visualizes the knowledge graph as a directed graph using NetworkX and Matplotlib. Nodes represent entities, and edges depict relationships with labels for clarity. The layout uses spring positioning with increased spacing for readability. Custom node colors and labeled edges enhance the visualization, displayed without axes.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

def wrap_text(text, max_words=8):
    """Wrap text if it contains more than `max_words` words."""
    words = text.split()
    return "Explanation" if len(words) > max_words else text

def visualize_graph_with_chains(kg, part_indices):
    """
    Visualize a directed graph highlighting nodes and edges by chain membership.

    Parameters:
        kg (list of tuples): The knowledge graph as a list of (node1, relation, node2).
        part_indices (dict): A dictionary where keys are chain names and values are lists of indices
                             corresponding to the `kg` entries in each chain.
    Returns:
        None
    """
    # Create graph
    G = nx.DiGraph()
    for node1, relation, node2 in kg:
        G.add_edge(node1, node2, label=relation)

    # Generate positions for the graph
    pos = nx.spring_layout(G, k=8, iterations=100, seed=0)

    # Define color maps
    chain_cmap = mcolors.LinearSegmentedColormap.from_list('chain_colors', ['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3', '#a6d854'])
    chain_norm = mcolors.Normalize(vmin=0, vmax=len(part_indices) - 1)

    # Initialize node and edge colors
    node_colors = ['lightblue'] * len(G.nodes())
    edge_colors = ['gray'] * len(G.edges())

    # Step 1: Assign distinct colors for each chain
    chain_color_map = {}
    for i, (chain_name, indices) in enumerate(part_indices.items()):
        color = chain_cmap(chain_norm(i))
        chain_color_map[chain_name] = color

    # Step 2: Color nodes based on the chain they belong to
    node_chain_map = {}
    for chain_name, indices in part_indices.items():
        color = chain_color_map[chain_name]
        for idx in indices:
            node1, relation, node2 = kg[idx]

            # Update node colors based on chain
            if node1 in G.nodes:
                node_chain_map[node1] = chain_name
                node_colors[list(G.nodes).index(node1)] = color
            if node2 in G.nodes:
                node_chain_map[node2] = chain_name
                node_colors[list(G.nodes).index(node2)] = color

    # Step 3: Assign edge colors based on the chain
    for i, (node1, node2) in enumerate(G.edges()):
        for chain_name, indices in part_indices.items():
            color = chain_color_map[chain_name]
            for idx in indices:
                n1, _, n2 = kg[idx]
                if (node1, node2) == (n1, n2):
                    edge_colors[i] = color
                    break

    # Apply label filtering
    wrapped_labels = {node: wrap_text(node) for node in G.nodes()}

    # Create the figure with subplots
    fig, axs = plt.subplots(1, 2, figsize=(20, 8), dpi=600)

    # Left: Original Knowledge Graph
    nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=1200, ax=axs[0])
    nx.draw_networkx_edges(G, pos, edge_color='gray', width=1.2, ax=axs[0])
    nx.draw_networkx_labels(G, pos, labels=wrapped_labels, font_size=6, ax=axs[0])
    edge_labels = nx.get_edge_attributes(G, 'label')
    wrapped_edge_labels = {edge: wrap_text(label) for edge, label in edge_labels.items()}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=wrapped_edge_labels, font_size=6, ax=axs[0])
    axs[0].set_title("Original Knowledge Graph", fontsize=10)
    axs[0].axis('off')

    # Right: Highlighted Nodes Based on Chains
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=1200, ax=axs[1], edgecolors='black')
    nx.draw_networkx_edges(G, pos, edge_color=edge_colors, width=1.5, ax=axs[1])
    nx.draw_networkx_labels(G, pos, labels=wrapped_labels, font_size=6, ax=axs[1])
    nx.draw_networkx_edge_labels(G, pos, edge_labels=wrapped_edge_labels, font_size=6, ax=axs[1])
    axs[1].set_title("Graph Highlighted by Chain Membership", fontsize=10)
    axs[1].axis('off')

    # Create a legend for chain colors
    handles = [plt.Line2D([0], [0], marker='o', color=color, markersize=10, linestyle='', label=chain_name)
               for chain_name, color in chain_color_map.items()]
    axs[1].legend(handles=handles, title="Chains", loc='upper right', fontsize=8)

    # Display the plot
    plt.show()

    # Print which nodes belong to which chain
    print("\n--- Node Chain Mapping ---")
    for node, chain in node_chain_map.items():
        print(f"Node '{node}' belongs to chain '{chain}'.")


In [ ]:
visualize_graph_with_chains(kg, part_indices)

Defines a function to perturb the knowledge graph by selectively removing triples belonging to specified parts. This allows testing the impact of missing information on downstream tasks or analysis. The function filters out triples associated with the indices of the parts to be removed and returns the modified knowledge graph.

In [ ]:
def perturb_kg_by_removing_parts(kg, parts_to_remove):
    """
    Perturbs the knowledge graph by removing triples from the specified parts.

    Parameters:
    - kg: The full knowledge graph triples list
    - parts_to_remove: List of part names to remove

    Returns:
    - perturbed_kg: The perturbed KG without the specified parts
    """
    perturbed_kg = []

    # Collect indices of the triples to keep based on parts to remove
    indices_to_remove = set()
    for part in parts_to_remove:
        indices_to_remove.update(part_indices[part])

    # Add triples that are not in the indices to remove
    perturbed_kg = [triple for i, triple in enumerate(kg) if i not in indices_to_remove]

    return perturbed_kg

This function computes the embedding for a given text using a specified model. It processes the text by removing newline characters and queries the OpenAI embeddings API to generate a vector representation, useful for similarity comparisons and downstream tasks.

In [ ]:
import random
import json
import unicodedata
import re
embedding_cache = {}
EMBEDDING_MODEL = "text-embedding-3-small"
def normalize_text(text):
    """
    Normalize text by removing excessive spaces, normalizing Unicode characters,
    and converting to lowercase.
    """
    text = text.replace("\n", " ").strip()  # Remove newlines and extra spaces
    text = unicodedata.normalize("NFKC", text)  # Normalize Unicode characters
    text = re.sub(r"\s+", " ", text)  # Replace multiple spaces with a single space
    text = text.lower()  # Convert to lowercase (optional but recommended)
    return text

def get_embedding(text):
    text = normalize_text(text)
    if text in embedding_cache:
        return embedding_cache[text]  # Return cached embedding
    embedding = client.embeddings.create(input=[text], model=EMBEDDING_MODEL).data[0].embedding
    embedding_cache[text] = embedding  # Store result in cache
    return embedding

Defines a function to query a GraphQAChain with a question and temperature setting, returning the answer and its embedding. The function initializes the chain with a specified graph and temperature, processes the question, and computes the embedding for the returned answer, facilitating downstream analysis or comparison

In [ ]:
def get_answer_and_embedding(question: str, temp: float, graph):
    """
    Sends a question and temperature to the GraphQAChain and returns the original answer string
    and its embedding as separate outputs.

    Args:
        question (str): The question to ask the chain.
        temp (float): The temperature setting for the OpenAI model.
        graph: The graph object for the GraphQAChain.

    Returns:
        Tuple[str, list]: The original answer as a string and its embedding as a list.
    """
    # Initialize the GraphQAChain with the specified temperature
    chain = GraphQAChain.from_llm(OpenAI(temperature=temp), graph=graph, verbose=False)

    # Run the question through the chain to get the answer
    original_answer = chain.run(question)
    original_answer_str = str(original_answer)

    # Compute the embedding for the original answer
    original_answer_embedding = get_embedding(original_answer)

    # Return both answer and embedding separately
    return original_answer_str, original_answer_embedding


This function visualizes the explainability of a knowledge graph by displaying the original graph and an enhanced graph with nodes and edges colored based on their importance coefficients. It leverages a directed graph structure, wraps node labels for readability, adjusts node sizes based on connectivity, and applies a custom colormap to represent the significance of graph components. The visualization is presented in a two-panel layout, highlighting both the original structure and the explainability features derived from Simple SMILE GraphRAG analysis. A color bar provides a reference for importance coefficients

In [ ]:
def plot_knowledge_graph_explainability(kg, part_indices, coeff):
    """
    Improved visualization of a knowledge graph with explainability features.

    Parameters:
        kg (list): Knowledge graph triplets (node1, relation, node2).
        part_indices (dict): Mapping of part names to indices.
        coeff (list): Importance coefficients for each part.
    """
    # Create graph
    G = nx.DiGraph()
    for node1, relation, node2 in kg:
        G.add_edge(node1, node2, label=relation)

    # Generate positions for the graph with increased spacing
    pos = nx.spring_layout(G, k=8, iterations=100, seed=0)

    # Create color map (blue for negative, red for positive)
    cmap = mcolors.LinearSegmentedColormap.from_list('red_blue', ['blue',  '#d3d3d3', 'red'])
    norm = mcolors.Normalize(vmin=-1, vmax=1)

    # Adjust node size based on degree (number of connections)
    node_sizes = [1500 + 100 * G.degree(node) for node in G.nodes()]

    # Assign node and edge colors based on the importance of each part
    node_colors = []
    edge_colors = []
    for node in G.nodes():
        for part_name, indices in part_indices.items():
            part_idx = int(part_name.split()[-1]) - 1
            coeff_value = coeff[part_idx]
            color = cmap(norm(coeff_value))
            if node in [kg[i][0] for i in indices] or node in [kg[i][2] for i in indices]:
                node_colors.append(color)
                break
        else:
            node_colors.append('#8da0cb')  # Default color if no match found

    for i, (node1, node2) in enumerate(G.edges()):
        for part_name, indices in part_indices.items():
            part_idx = int(part_name.split()[-1]) - 1
            coeff_value = coeff[part_idx]
            color = cmap(norm(coeff_value))
            if i in indices:
                edge_colors.append(color)
                break
        else:
            edge_colors.append('gray')  # Default color if no match found

    # Wrap the text labels
    wrapped_labels = {node: wrap_text(node) for node in G.nodes()}

    # Create the figure with subplots
    fig, axs = plt.subplots(1, 2, figsize=(24, 10), dpi=600, gridspec_kw={'width_ratios': [1, 1.3]})

    # Left: Original Knowledge Graph
    nx.draw_networkx_nodes(G, pos, node_color= '#d3d3d3', node_size=node_sizes, ax=axs[0])
    nx.draw_networkx_edges(G, pos, edge_color='gray', width=1.5, ax=axs[0])
    nx.draw_networkx_labels(G, pos, labels=wrapped_labels, font_size=8, ax=axs[0])
    edge_labels = nx.get_edge_attributes(G, 'label')
    wrapped_edge_labels = {edge: wrap_text(label) for edge, label in edge_labels.items()}
    nx.draw_networkx_edge_labels(G, pos, edge_labels=wrapped_edge_labels, font_size=8, ax=axs[0])
    axs[0].set_title("Original Knowledge Graph", fontsize=12)
    axs[0].axis('off')

    # Right: SMILE Explainability with Node and Edge Colors
    nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=node_sizes, ax=axs[1])
    nx.draw_networkx_edges(G, pos, edge_color=edge_colors, width=1.8, ax=axs[1])
    nx.draw_networkx_labels(G, pos, labels=wrapped_labels, font_size=8, ax=axs[1])
    nx.draw_networkx_edge_labels(G, pos, edge_labels=wrapped_edge_labels, font_size=8, ax=axs[1])
    axs[1].set_title("Simple SMILE GraphRAG Explainability", fontsize=12)
    axs[1].axis('off')

    # Show the color bar
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    fig.colorbar(sm, ax=axs[1], label='Importance Coefficients')

    # Save the figure
    plt.savefig('knowledge_graph_explainability_improved.png', bbox_inches='tight')

    # Display the plot
    plt.show()


Defines the question to query the GraphQAChain or knowledge retrieval system. Here, the question seeks information about it, a framework that integrates external knowledge bases to improve the accuracy and reliability of AI-generated responses

In [ ]:
question = "what is Network_Security_Services?"
#Portion 3

This snippet sets the temperature parameter to 0 for deterministic response generation and queries the GraphQAChain with the question. The function get_answer_and_embedding returns the original answer as a string along with its embedding. The answer is then printed for review.


In [ ]:
# Initialize the LLM
llm = OpenAI(temperature=0)
temp = 0
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(original_answer_str)

In [ ]:
# Define the original vector (all parts present)
original = np.array([1, 1, 1, 1, 1,1, 1, 1, 1, 1])
original = original.reshape(1, -1)  # Shape becomes (1, 10)

This function calculates the importance coefficients for perturbations on a knowledge graph while providing
detailed logging at each iteration. It removes random parts of the knowledge graph, generates perturbed
responses, computes similarities to the original answer, and fits a linear regression model to calculate
coefficients. The function includes:

- Temperature (`temp`) parameter to adjust the behavior of the GraphQAChain.
- Iterative logs showing the removed parts, perturbed responses, and calculated similarities.
- A summary of all similarities and weights after processing.
The coefficients provide insights into the contribution of each part of the knowledge graph to response fidelity.

In [ ]:
def calculate_coefficients_print_Temerature(temp, original, kg, part_names, question, original_answer_embedding, original_answer_str):
    """
    Function to calculate coefficients for perturbations on a knowledge graph.
    It removes parts of the KG, generates perturbed responses, and calculates coefficients.

    Parameters:
    - temp: Temperature value (0 or 1)
    - original: Original vector (numpy array)
    - kg: Knowledge graph (list of triples)
    - part_names: List of part names in the KG
    - question: Question for GraphQAChain
    - original_answer_embedding: Embedding of the original answer
    - original_answer_str: Original answer text

    Returns:
    - coeff: Coefficients from linear regression
    """
        # Define the original vector (all parts present)
    original = np.array([1, 1, 1, 1, 1,1, 1, 1, 1, 1])
    original = original.reshape(1, -1)  # Shape becomes (1, 10)
    similarities_wd = []
    perturbations_vect2 = []
    perturbation_texts = []
    generated_embeddings = []
    epsilon = 1e-6

    for i in range(20):  # Number of perturbations
        perturbation_vector = original.copy().flatten()
        num_parts_to_remove = random.randint(1, len(part_names))
        parts_to_remove_indices = random.sample(range(len(part_names)), num_parts_to_remove)

        for part_idx in parts_to_remove_indices:
            perturbation_vector[part_idx] = 0

        perturbations_vect2.append(perturbation_vector)
        parts_to_remove = [part_names[idx] for idx in parts_to_remove_indices]

        # Call the perturb_kg_by_removing_parts function directly
        perturbed_kg = perturb_kg_by_removing_parts(kg, parts_to_remove)

        graph_temp = NetworkxEntityGraph()

        for (node1, relation, node2) in perturbed_kg:
            graph_temp.add_triple(KnowledgeTriple(node1, relation, node2))

        chain = GraphQAChain.from_llm(OpenAI(temperature=temp), graph=graph_temp, verbose=False)
        temp_response = chain.run(question)

        perturbation_texts.append(temp_response)

        # Call the get_embedding function directly
        temp_response_embedding = get_embedding(temp_response)
        generated_embeddings.append(temp_response_embedding)

        # Calculate Wasserstein distance
        similarity_wd = wasserstein_distance(original_answer_embedding, temp_response_embedding)
        similarities_wd.append(similarity_wd)

        # Print progress for each iteration
        print(f"Iteration {i + 1}")  # Fixed to use `i` defined in the loop
        print(f"Parts removed: {parts_to_remove}")
        print(f"original_answer response: {original_answer_str}")
        print(f"Perturbed response: {temp_response}")
        print(f"Distancee with original answer: {similarity_wd}\n")

    perturbations_vect2 = np.array(perturbations_vect2)
    distances = sklearn.metrics.pairwise_distances(perturbations_vect2, original, metric='cosine').ravel()

    kernel_width = 0.25
    weights = np.sqrt(np.exp(-(distances**2) / kernel_width**2))

    # Print all similarities and weights
    print(f"similarities_wd: {similarities_wd}")
    print(f"Weights: {weights}")




    # Calculate inverse Wasserstein distances
    inverse_similarities_wd = [1.0 / (dist + epsilon) for dist in similarities_wd]

    # Scale inverse Wasserstein distances
    min_value = min(inverse_similarities_wd)
    max_value = max(inverse_similarities_wd)
    if min_value == max_value:
      print("Warning: min_value and max_value are equal. Avoiding division by zero.")
      scaled_inverse_similarities_wd = [1.0 for _ in inverse_similarities_wd]  # Assign a constant
    else:
      scaled_inverse_similarities_wd = [
          (value - min_value) / (max_value - min_value) for value in inverse_similarities_wd
      ]
    # Linear regression for cosine similarities
    simpler_model = LinearRegression()
    # Linear regression for scaled inverse Wasserstein distances
    simpler_model.fit(X=perturbations_vect2, y=scaled_inverse_similarities_wd, sample_weight=weights)
    coeff = simpler_model.coef_
    return coeff


# Stability: Consistent explanations despite small changes in input or graph structure.

This script augments the knowledge graph ('kg') with an additional triple to assess stability
in response generation and explainability. The stability analysis involves perturbing the knowledge
graph by removing random parts over 20 iterations for each temperature setting (0 and 1).

Key steps for stability analysis:
1. **Augmentation:** Adds a triple to the knowledge graph to introduce a new component for stability assessment.
2. **Perturbations:** Randomly removes parts of the knowledge graph multiple times to analyze the system's
   ability to maintain consistent and meaningful responses.
3. **Temperature Variation:** Runs the process with both deterministic (temperature = 0) and stochastic
   (temperature = 1) configurations to observe the impact of randomness on stability.
4. **Stability Metric:** Evaluates response consistency and similarity under perturbations, providing insights
   into the robustness of the system in preserving the core knowledge structure and response fidelity.

The results highlight how stable and resilient the system is to changes in the underlying knowledge graph.

In [ ]:
def wrap_label(label, width=15):
    """Wraps labels to fit within a specified width for better visualization."""
    return '\n'.join(textwrap.wrap(label, width))

def wrap_text(node1, relation, node2, max_words=8):
    """Wrap text if it contains more than `max_words` words and append '_explanation' to abstract nodes."""
    if relation == "abstract":
        words = node2.split()
        return f"{node1}_explanation" if len(words) > max_words else node2
    return node2

def build_graph(kg, coeff, part_indices):
    """Helper function to build graph, assign colors, and sizes."""
    G = nx.DiGraph()
    for node1, relation, node2 in kg:
        wrapped_node1 = wrap_label(node1)
        wrapped_node2 = wrap_label(wrap_text(node1, relation, node2))
        wrapped_relation = wrap_label(relation)
        G.add_edge(wrapped_node1, wrapped_node2, label=wrapped_relation)

    pos = nx.spring_layout(G, k=8, iterations=100, seed=0)
    cmap = mcolors.LinearSegmentedColormap.from_list('red_blue', ['blue', '#d3d3d3', 'red'])
    norm = mcolors.Normalize(vmin=-1, vmax=1)
    node_sizes = [1500 + 100 * G.degree(node) for node in G.nodes()]

    node_colors = []
    for node in G.nodes():
        for part_name, indices in part_indices.items():
            part_idx = int(part_name.split()[-1]) - 1
            coeff_value = coeff[part_idx]
            color = cmap(norm(coeff_value))
            if any(i < len(kg) and (wrap_label(node) == wrap_label(kg[i][0]) or wrap_label(node) == wrap_label(wrap_text(kg[i][0], kg[i][1], kg[i][2]))) for i in indices):
                node_colors.append(color)
                break
        else:
            node_colors.append('#8da0cb')

    edge_colors = []
    for i, (node1, node2) in enumerate(G.edges()):
        for part_name, indices in part_indices.items():
            part_idx = int(part_name.split()[-1]) - 1
            coeff_value = coeff[part_idx]
            color = cmap(norm(coeff_value))
            if i in indices:
                edge_colors.append(color)
                break
        else:
            edge_colors.append('gray')

    return G, pos, node_sizes, node_colors, edge_colors

def plot_knowledge_graph_explainability_compare(kg_original, coeff_original, part_indices_original,
                                                 kg_added, coeff_added, part_indices_added):
    """
    Visualize two knowledge graphs (original and added) side by side with explainability features.
    """
    G_original, pos_original, node_sizes_original, node_colors_original, edge_colors_original = build_graph(
        kg_original, coeff_original, part_indices_original)
    G_added, pos_added, node_sizes_added, node_colors_added, edge_colors_added = build_graph(
        kg_added, coeff_added, part_indices_added)

    fig, axs = plt.subplots(1, 2, figsize=(24, 10), dpi=300)

    nx.draw_networkx_nodes(G_original, pos_original, node_color=node_colors_original, node_size=node_sizes_original, ax=axs[0])
    nx.draw_networkx_edges(G_original, pos_original, edge_color=edge_colors_original, width=1.5, ax=axs[0])
    nx.draw_networkx_labels(G_original, pos_original, font_size=8, ax=axs[0])
    edge_labels_original = nx.get_edge_attributes(G_original, 'label')
    nx.draw_networkx_edge_labels(G_original, pos_original, edge_labels=edge_labels_original, font_size=8, ax=axs[0])
    axs[0].set_title("Original Knowledge Graph", fontsize=14)
    axs[0].axis('off')

    nx.draw_networkx_nodes(G_added, pos_added, node_color=node_colors_added, node_size=node_sizes_added, ax=axs[1])
    nx.draw_networkx_edges(G_added, pos_added, edge_color=edge_colors_added, width=1.5, ax=axs[1])
    nx.draw_networkx_labels(G_added, pos_added, font_size=8, ax=axs[1])
    edge_labels_added = nx.get_edge_attributes(G_added, 'label')
    nx.draw_networkx_edge_labels(G_added, pos_added, edge_labels=edge_labels_added, font_size=8, ax=axs[1])
    axs[1].set_title("Added Knowledge Graph", fontsize=14)
    axs[1].axis('off')

    cmap = mcolors.LinearSegmentedColormap.from_list('red_blue', ['blue', '#d3d3d3', 'red'])
    norm = mcolors.Normalize(vmin=-1, vmax=1)
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
    sm.set_array([])
    fig.colorbar(sm, ax=axs, orientation='horizontal', label='Importance Coefficients', fraction=0.03, pad=0.05)

    plt.savefig('knowledge_graph_explainability_comparison.png', bbox_inches='tight')
    plt.show()


In [ ]:
def calculate_jaccard_index(coeff_original, coeff_added):
    """
    Calculate the Jaccard index (intersection over union) between two coefficient arrays.

    Parameters:
    coeff_original (np.ndarray): Original coefficients.
    coeff_added (np.ndarray): Added coefficients.

    Returns:
    float: The Jaccard index.
    """
    intersection = np.minimum(coeff_original, coeff_added).sum()
    union = np.maximum(coeff_original, coeff_added).sum()
    jaccard_index = intersection / union
    return jaccard_index

In [ ]:
question = "what is Network_Security_Services?"
#Part 3

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 1) by introducing small perturbations in the input or graph structure -(Scooby-Doo_and_the_Cyber_Chase_(video_game)) → (platform) → (PlayStation, Game Boy Advance)- and assessing the consistency of generated explanations.

In [ ]:
kg_added = [
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "type", "Software"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "label", "Scooby-Doo and the Cyber Chase (video game)"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "abstract", "Scooby-Doo and the Cyber Chase is a Scooby-Doo video game based on the Warner Brothers film Scooby-Doo and the Cyber Chase. The game was released for the PlayStation and Game Boy Advance in 2001. The PlayStation version became a 'Greatest Hits' title in 2003."),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)" ,"platform", "PlayStation, Game Boy Advance"),

    ("Mirai_(malware)", "type", "Software"),
    ("Mirai_(malware)", "label", "Mirai (malware)"),
    ("Mirai_(malware)", "abstract", "Mirai (from the Japanese word for 'future', 未来) is a malware that turns networked devices running Linux into remotely controlled bots that can be used as part of a botnet in large-scale network attacks. It primarily targets online consumer devices such as IP cameras and home routers. The Mirai botnet was first found in August 2016 by MalwareMustDie, a white hat malware research group, and has been used in some of the largest and most disruptive distributed denial of service (DDoS) attacks, including an attack on 20 September 2016 on computer security journalist Brian Krebs' website, an attack on French web host OVH, and the October 2016 Dyn cyberattack. According to a chat log between Anna-senpai and Robert Coelho, Mirai was named after the 2011 TV anime series Mirai Nikki. The software was initially used by the creators to DDoS Minecraft servers and companies offering DDoS protection to Minecraft servers, with the authors using Mirai to operate a protection racket. The source code for Mirai was subsequently published on Hack Forums as open-source. Since the source code was published, the techniques have been adapted in other malware projects."),

    ("Network_Security_Services", "type", "Software"),
    ("Network_Security_Services", "label", "Network Security Services"),
    ("Network_Security_Services", "abstract", "Network Security Services (NSS) is a collection of cryptographic computer libraries designed to support cross-platform development of security-enabled client and server applications with optional support for hardware TLS/SSL acceleration on the server side and hardware smart cards on the client side. NSS provides a complete open-source implementation of cryptographic libraries supporting Transport Layer Security (TLS) / Secure Sockets Layer (SSL) and S/MIME. NSS releases prior to version 3.14 are tri-licensed under the Mozilla Public License 1.1, the GNU General Public License, and the GNU Lesser General Public License. Since release 3.14, NSS releases are licensed under GPL-compatible Mozilla Public License 2.0."),

    ("MS_Antivirus_(malware)", "type", "Software"),
    ("MS_Antivirus_(malware)", "label", "MS Antivirus (malware)"),
    ("MS_Antivirus_(malware)", "abstract", "MS Antivirus (also known as Spyware Protect 2009 and Antivirus XP 2008/Antivirus2009/SecurityTool/etc) is a scareware rogue anti-virus which purports to remove virus infections found on a computer running Microsoft Windows. It attempts to scam the user into purchasing a 'full version' of the software. The company and the individuals behind Bakasoftware operated under other different 'company' names, including Innovagest2000, Innovative Marketing Ukraine, Pandora Software, LocusSoftware, etc."),

    ("CyberCIEGE", "type", "Software"),
    ("CyberCIEGE", "label", "CyberCIEGE"),
    ("CyberCIEGE", "abstract", "CyberCIEGE is a serious game designed to teach network security concepts. Its development was sponsored by the U.S. Navy, and it is used as a training tool by agencies of the U.S. government, universities and community colleges. CyberCIEGE covers a broad range of cybersecurity topics. Players purchase and configure computers and network devices to keep demanding users happy (e.g., by providing Internet access) all while protecting assets from a variety of attacks. The game includes a number of different scenarios, some of which focus on basic training and awareness, others on more advanced network security concepts."),

    ("CyberExtension", "type", "Software"),
    ("CyberExtension", "label", "CyberExtension"),
    ("CyberExtension", "abstract", "CyberExtension is a managed virtual learning environment built by Right Reason Technologies (RRT) that is designed to be used as an online extension of a school district in the USA. The system is currently used for homebound students, credit recovery and supplemental coursework, and has been successfully deployed to help students with 'school phobia' and students undergoing medical treatment."),

    ("CyberGladiators", "type", "Software"),
    ("CyberGladiators", "label", "CyberGladiators"),
    ("CyberGladiators", "abstract", "CyberGladiators is a 1996 fighting game developed by K.A.A. (label of Dynamix) and published by Sierra On-Line. The game is divided of fights between members of the Quaaflax Alliance and members of the terrorist organization Gy Djin. All characters have been transformed by a cosmic storm into CyberGladiators."),

    ("CyberLink_MediaShow", "type", "Software"),
    ("CyberLink_MediaShow", "label", "CyberLink MediaShow"),
    ("CyberLink_MediaShow", "abstract", "CyberLink MediaShow is a software application for organizing, editing and sharing photos and videos, published by CyberLink Corporation. MediaShow allows users to import video and photo files from digital cameras, phones, and camcorders. Its main competitors are Arcsoft MediaImpression, Adobe Bridge, Roxio Creator and Google Picasa."),

    ("CyberQuery", "type", "Software"),
    ("CyberQuery", "label", "CyberQuery"),
    ("CyberQuery", "abstract", "Cyberquery is a software product of Cyberscience Corporation Inc. Originally developed for data handling and analysis on Data General AOS and AOS/VS minicomputers, then the available platforms for Cyberquery were extended to all major UNIX platforms, OpenVMS, and Microsoft Windows."),

    ("CyberStrike", "type", "Software"),
    ("CyberStrike", "label", "CyberStrike"),
    ("CyberStrike", "abstract", "CyberStrike is a futuristic 3D combat online game by Simutronics, involving team combat between customizable mechs, each of which is controlled by a different player. Initially exclusive to the GEnie online service, it opened in February 1993, and later that year it caused Computer Gaming World to create the new category of 'Online Game of the Year' so it could be awarded to CyberStrike. In 1994, it was offered in stores by MicroProse. In 1997, a sequel, Cyberstrike 2, was announced as a joint project between Simutronics and Sony's 989 Studios.")
]

part_indices_added ={
"Part 1": range(0, 3),
"Part 2": range(3, 6),
"Part 3": range(6, 9),
"Part 4": range(9, 12),
"Part 5": range(12, 15),
"Part 6": range(15, 18),
"Part 7": range(18, 21),
"Part 8": range(21, 24),
"Part 9": range(24, 27),
"Part 10": range(27, 30)
}
part_names_added = list(part_indices_added.keys())


In [ ]:
print("Original KG node count:", len(set(node for triple in kg for node in (triple[0], triple[2]))))
print("Updated KG node count:", len(set(node for triple in kg_added for node in (triple[0], triple[2]))))

In [ ]:
visualize_graph_with_chains(kg_added, part_indices_added)

In [ ]:
temp = 1
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(f"Original answer: {original_answer_str}")

In [ ]:
temp= 1
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff = np.round(coeff, 3)  # Rounds to 3 decimal places
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.074, -0.349,  0.278, -0.24 , -0.256,  0.23 ,  0.057, -0.081,
        0.373, -0.109])
coeff_added =np.array([ 0.175,  0.01 ,  0.289,  0.231, -0.192,  0.171,  0.049, -0.006,
       -0.025, -0.465])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 0) by introducing small perturbations in the input or graph structure -(Scooby-Doo_and_the_Cyber_Chase_(video_game)) → (platform) → (PlayStation, Game Boy Advance)- and assessing the consistency of generated explanations.

In [ ]:
temp = 0
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(f"Original answer: {original_answer_str}")

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff = np.round(coeff, 3)  # Rounds to 3 decimal places
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.,  0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0.])
coeff_added = np.array([ 0.,  0.,  1., -0.,  0.,  0., -0., -0.,  0.,  0.])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 1) by introducing small perturbations in the input or graph structure -(Mirai_(malware)) → (target) → (IoT Devices)- and assessing the consistency of generated explanations.

In [ ]:
kg_added = [
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "type", "Software"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "label", "Scooby-Doo and the Cyber Chase (video game)"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "abstract", "Scooby-Doo and the Cyber Chase is a Scooby-Doo video game based on the Warner Brothers film Scooby-Doo and the Cyber Chase. The game was released for the PlayStation and Game Boy Advance in 2001. The PlayStation version became a 'Greatest Hits' title in 2003."),

    ("Mirai_(malware)", "type", "Software"),
    ("Mirai_(malware)", "label", "Mirai (malware)"),
    ("Mirai_(malware)", "abstract", "Mirai (from the Japanese word for 'future', 未来) is a malware that turns networked devices running Linux into remotely controlled bots that can be used as part of a botnet in large-scale network attacks. It primarily targets online consumer devices such as IP cameras and home routers. The Mirai botnet was first found in August 2016 by MalwareMustDie, a white hat malware research group, and has been used in some of the largest and most disruptive distributed denial of service (DDoS) attacks, including an attack on 20 September 2016 on computer security journalist Brian Krebs' website, an attack on French web host OVH, and the October 2016 Dyn cyberattack. According to a chat log between Anna-senpai and Robert Coelho, Mirai was named after the 2011 TV anime series Mirai Nikki. The software was initially used by the creators to DDoS Minecraft servers and companies offering DDoS protection to Minecraft servers, with the authors using Mirai to operate a protection racket. The source code for Mirai was subsequently published on Hack Forums as open-source. Since the source code was published, the techniques have been adapted in other malware projects."),
    ("Mirai_(malware)","target","IoT Devices"),

    ("Network_Security_Services", "type", "Software"),
    ("Network_Security_Services", "label", "Network Security Services"),
    ("Network_Security_Services", "abstract", "Network Security Services (NSS) is a collection of cryptographic computer libraries designed to support cross-platform development of security-enabled client and server applications with optional support for hardware TLS/SSL acceleration on the server side and hardware smart cards on the client side. NSS provides a complete open-source implementation of cryptographic libraries supporting Transport Layer Security (TLS) / Secure Sockets Layer (SSL) and S/MIME. NSS releases prior to version 3.14 are tri-licensed under the Mozilla Public License 1.1, the GNU General Public License, and the GNU Lesser General Public License. Since release 3.14, NSS releases are licensed under GPL-compatible Mozilla Public License 2.0."),

    ("MS_Antivirus_(malware)", "type", "Software"),
    ("MS_Antivirus_(malware)", "label", "MS Antivirus (malware)"),
    ("MS_Antivirus_(malware)", "abstract", "MS Antivirus (also known as Spyware Protect 2009 and Antivirus XP 2008/Antivirus2009/SecurityTool/etc) is a scareware rogue anti-virus which purports to remove virus infections found on a computer running Microsoft Windows. It attempts to scam the user into purchasing a 'full version' of the software. The company and the individuals behind Bakasoftware operated under other different 'company' names, including Innovagest2000, Innovative Marketing Ukraine, Pandora Software, LocusSoftware, etc."),

    ("CyberCIEGE", "type", "Software"),
    ("CyberCIEGE", "label", "CyberCIEGE"),
    ("CyberCIEGE", "abstract", "CyberCIEGE is a serious game designed to teach network security concepts. Its development was sponsored by the U.S. Navy, and it is used as a training tool by agencies of the U.S. government, universities and community colleges. CyberCIEGE covers a broad range of cybersecurity topics. Players purchase and configure computers and network devices to keep demanding users happy (e.g., by providing Internet access) all while protecting assets from a variety of attacks. The game includes a number of different scenarios, some of which focus on basic training and awareness, others on more advanced network security concepts."),

    ("CyberExtension", "type", "Software"),
    ("CyberExtension", "label", "CyberExtension"),
    ("CyberExtension", "abstract", "CyberExtension is a managed virtual learning environment built by Right Reason Technologies (RRT) that is designed to be used as an online extension of a school district in the USA. The system is currently used for homebound students, credit recovery and supplemental coursework, and has been successfully deployed to help students with 'school phobia' and students undergoing medical treatment."),

    ("CyberGladiators", "type", "Software"),
    ("CyberGladiators", "label", "CyberGladiators"),
    ("CyberGladiators", "abstract", "CyberGladiators is a 1996 fighting game developed by K.A.A. (label of Dynamix) and published by Sierra On-Line. The game is divided of fights between members of the Quaaflax Alliance and members of the terrorist organization Gy Djin. All characters have been transformed by a cosmic storm into CyberGladiators."),

    ("CyberLink_MediaShow", "type", "Software"),
    ("CyberLink_MediaShow", "label", "CyberLink MediaShow"),
    ("CyberLink_MediaShow", "abstract", "CyberLink MediaShow is a software application for organizing, editing and sharing photos and videos, published by CyberLink Corporation. MediaShow allows users to import video and photo files from digital cameras, phones, and camcorders. Its main competitors are Arcsoft MediaImpression, Adobe Bridge, Roxio Creator and Google Picasa."),

    ("CyberQuery", "type", "Software"),
    ("CyberQuery", "label", "CyberQuery"),
    ("CyberQuery", "abstract", "Cyberquery is a software product of Cyberscience Corporation Inc. Originally developed for data handling and analysis on Data General AOS and AOS/VS minicomputers, then the available platforms for Cyberquery were extended to all major UNIX platforms, OpenVMS, and Microsoft Windows."),

    ("CyberStrike", "type", "Software"),
    ("CyberStrike", "label", "CyberStrike"),
    ("CyberStrike", "abstract", "CyberStrike is a futuristic 3D combat online game by Simutronics, involving team combat between customizable mechs, each of which is controlled by a different player. Initially exclusive to the GEnie online service, it opened in February 1993, and later that year it caused Computer Gaming World to create the new category of 'Online Game of the Year' so it could be awarded to CyberStrike. In 1994, it was offered in stores by MicroProse. In 1997, a sequel, Cyberstrike 2, was announced as a joint project between Simutronics and Sony's 989 Studios.")
]
part_indices_added = {
    "Part 1": range(0, 3),   # Scooby-Doo_and_the_Cyber_Chase_(video_game)
    "Part 2": range(3, 7),   # Mirai_(malware) (has an extra 'target' attribute)
    "Part 3": range(7, 10),  # Network_Security_Services
    "Part 4": range(10, 13), # MS_Antivirus_(malware)
    "Part 5": range(13, 16), # CyberCIEGE
    "Part 6": range(16, 19), # CyberExtension
    "Part 7": range(19, 22), # CyberGladiators
    "Part 8": range(22, 25), # CyberLink_MediaShow
    "Part 9": range(25, 28), # CyberQuery
    "Part 10": range(28, 31) # CyberStrike
}

part_names_added = list(part_indices_added.keys())


In [ ]:
visualize_graph_with_chains(kg_added, part_indices_added)

In [ ]:
temp = 1
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(original_answer_str)

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.074, -0.349,  0.278, -0.24 , -0.256,  0.23 ,  0.057, -0.081,
        0.373, -0.109])
coeff_added = np.array([-0.19716259,  0.13965323,  0.37876745, -0.31036451,  0.36941792,
        0.54564323, -0.14606655, -0.18255853, -0.64883957, -0.47013287])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 0) by introducing small perturbations in the input or graph structure -(Mirai_(malware)) → (target) → (IoT Devices)- and assessing the consistency of generated explanations.

In [ ]:
temp = 0
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(f"Original answer: {original_answer_str}")

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.,  0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0.])
coeff_added =np.array([-0.19981933,  0.28225844,  0.38885523,  0.38597305, -0.34761973,
        0.25943983,  0.22293793, -0.52873037,  0.33054272, -0.45729486])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 1) by introducing small perturbations in the input or graph structure -(Network_Security_Services) → (developer) → (Mozilla)- and assessing the consistency of generated explanations.

In [ ]:
kg_added = [
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "type", "Software"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "label", "Scooby-Doo and the Cyber Chase (video game)"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "abstract", "Scooby-Doo and the Cyber Chase is a Scooby-Doo video game based on the Warner Brothers film Scooby-Doo and the Cyber Chase. The game was released for the PlayStation and Game Boy Advance in 2001. The PlayStation version became a 'Greatest Hits' title in 2003."),

    ("Mirai_(malware)", "type", "Software"),
    ("Mirai_(malware)", "label", "Mirai (malware)"),
    ("Mirai_(malware)", "abstract", "Mirai (from the Japanese word for 'future', 未来) is a malware that turns networked devices running Linux into remotely controlled bots that can be used as part of a botnet in large-scale network attacks. It primarily targets online consumer devices such as IP cameras and home routers. The Mirai botnet was first found in August 2016 by MalwareMustDie, a white hat malware research group, and has been used in some of the largest and most disruptive distributed denial of service (DDoS) attacks, including an attack on 20 September 2016 on computer security journalist Brian Krebs' website, an attack on French web host OVH, and the October 2016 Dyn cyberattack. According to a chat log between Anna-senpai and Robert Coelho, Mirai was named after the 2011 TV anime series Mirai Nikki. The software was initially used by the creators to DDoS Minecraft servers and companies offering DDoS protection to Minecraft servers, with the authors using Mirai to operate a protection racket. The source code for Mirai was subsequently published on Hack Forums as open-source. Since the source code was published, the techniques have been adapted in other malware projects."),

    ("Network_Security_Services", "type", "Software"),
    ("Network_Security_Services", "label", "Network Security Services"),
    ("Network_Security_Services", "abstract", "Network Security Services (NSS) is a collection of cryptographic computer libraries designed to support cross-platform development of security-enabled client and server applications with optional support for hardware TLS/SSL acceleration on the server side and hardware smart cards on the client side. NSS provides a complete open-source implementation of cryptographic libraries supporting Transport Layer Security (TLS) / Secure Sockets Layer (SSL) and S/MIME. NSS releases prior to version 3.14 are tri-licensed under the Mozilla Public License 1.1, the GNU General Public License, and the GNU Lesser General Public License. Since release 3.14, NSS releases are licensed under GPL-compatible Mozilla Public License 2.0."),
    ("Network_Security_Services", "developer" , "Mozilla"),

    ("MS_Antivirus_(malware)", "type", "Software"),
    ("MS_Antivirus_(malware)", "label", "MS Antivirus (malware)"),
    ("MS_Antivirus_(malware)", "abstract", "MS Antivirus (also known as Spyware Protect 2009 and Antivirus XP 2008/Antivirus2009/SecurityTool/etc) is a scareware rogue anti-virus which purports to remove virus infections found on a computer running Microsoft Windows. It attempts to scam the user into purchasing a 'full version' of the software. The company and the individuals behind Bakasoftware operated under other different 'company' names, including Innovagest2000, Innovative Marketing Ukraine, Pandora Software, LocusSoftware, etc."),

    ("CyberCIEGE", "type", "Software"),
    ("CyberCIEGE", "label", "CyberCIEGE"),
    ("CyberCIEGE", "abstract", "CyberCIEGE is a serious game designed to teach network security concepts. Its development was sponsored by the U.S. Navy, and it is used as a training tool by agencies of the U.S. government, universities and community colleges. CyberCIEGE covers a broad range of cybersecurity topics. Players purchase and configure computers and network devices to keep demanding users happy (e.g., by providing Internet access) all while protecting assets from a variety of attacks. The game includes a number of different scenarios, some of which focus on basic training and awareness, others on more advanced network security concepts."),

    ("CyberExtension", "type", "Software"),
    ("CyberExtension", "label", "CyberExtension"),
    ("CyberExtension", "abstract", "CyberExtension is a managed virtual learning environment built by Right Reason Technologies (RRT) that is designed to be used as an online extension of a school district in the USA. The system is currently used for homebound students, credit recovery and supplemental coursework, and has been successfully deployed to help students with 'school phobia' and students undergoing medical treatment."),

    ("CyberGladiators", "type", "Software"),
    ("CyberGladiators", "label", "CyberGladiators"),
    ("CyberGladiators", "abstract", "CyberGladiators is a 1996 fighting game developed by K.A.A. (label of Dynamix) and published by Sierra On-Line. The game is divided of fights between members of the Quaaflax Alliance and members of the terrorist organization Gy Djin. All characters have been transformed by a cosmic storm into CyberGladiators."),

    ("CyberLink_MediaShow", "type", "Software"),
    ("CyberLink_MediaShow", "label", "CyberLink MediaShow"),
    ("CyberLink_MediaShow", "abstract", "CyberLink MediaShow is a software application for organizing, editing and sharing photos and videos, published by CyberLink Corporation. MediaShow allows users to import video and photo files from digital cameras, phones, and camcorders. Its main competitors are Arcsoft MediaImpression, Adobe Bridge, Roxio Creator and Google Picasa."),

    ("CyberQuery", "type", "Software"),
    ("CyberQuery", "label", "CyberQuery"),
    ("CyberQuery", "abstract", "Cyberquery is a software product of Cyberscience Corporation Inc. Originally developed for data handling and analysis on Data General AOS and AOS/VS minicomputers, then the available platforms for Cyberquery were extended to all major UNIX platforms, OpenVMS, and Microsoft Windows."),

    ("CyberStrike", "type", "Software"),
    ("CyberStrike", "label", "CyberStrike"),
    ("CyberStrike", "abstract", "CyberStrike is a futuristic 3D combat online game by Simutronics, involving team combat between customizable mechs, each of which is controlled by a different player. Initially exclusive to the GEnie online service, it opened in February 1993, and later that year it caused Computer Gaming World to create the new category of 'Online Game of the Year' so it could be awarded to CyberStrike. In 1994, it was offered in stores by MicroProse. In 1997, a sequel, Cyberstrike 2, was announced as a joint project between Simutronics and Sony's 989 Studios.")
]
part_indices_added = {
    "Part 1": range(0, 3),   # Scooby-Doo_and_the_Cyber_Chase_(video_game)
    "Part 2": range(3, 7),   # Mirai_(malware) (has an extra 'target' attribute)
    "Part 3": range(7, 10),  # Network_Security_Services
    "Part 4": range(10, 13), # MS_Antivirus_(malware)
    "Part 5": range(13, 16), # CyberCIEGE
    "Part 6": range(16, 19), # CyberExtension
    "Part 7": range(19, 22), # CyberGladiators
    "Part 8": range(22, 25), # CyberLink_MediaShow
    "Part 9": range(25, 28), # CyberQuery
    "Part 10": range(28, 31) # CyberStrike
}

part_names_added = list(part_indices_added.keys())


In [ ]:
visualize_graph_with_chains(kg, part_indices)

In [ ]:
temp = 1
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(original_answer_str)

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.074, -0.349,  0.278, -0.24 , -0.256,  0.23 ,  0.057, -0.081,
        0.373, -0.109])
coeff_added = np.array([ 0.0770485 ,  0.05072621, -0.14007454, -0.4948971 ,  0.0377721 ,
        0.33359353,  0.10208947,  0.06688596,  0.29672679,  0.24337406])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 0) by introducing small perturbations in the input or graph structure -(Network_Security_Services) → (developer) → (Mozilla)- and assessing the consistency of generated explanations.

In [ ]:
temp = 0
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(f"Original answer: {original_answer_str}")

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.,  0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0.])
coeff_added = np.array([-0.31144466, -0.01937649,  0.57757609, -0.46581808,  0.09925493,
       -0.30095422,  0.00213296,  0.08485401,  0.14835597, -0.01544272])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 1) by introducing small perturbations in the input or graph structure -(MS_Antivirus_(malware)) → (developer) → (Bakasoftware)- and assessing the consistency of generated explanations.

In [ ]:
kg_added = [
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "type", "Software"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "label", "Scooby-Doo and the Cyber Chase (video game)"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "abstract", "Scooby-Doo and the Cyber Chase is a Scooby-Doo video game based on the Warner Brothers film Scooby-Doo and the Cyber Chase. The game was released for the PlayStation and Game Boy Advance in 2001. The PlayStation version became a 'Greatest Hits' title in 2003."),

    ("Mirai_(malware)", "type", "Software"),
    ("Mirai_(malware)", "label", "Mirai (malware)"),
    ("Mirai_(malware)", "abstract", "Mirai (from the Japanese word for 'future', 未来) is a malware that turns networked devices running Linux into remotely controlled bots that can be used as part of a botnet in large-scale network attacks. It primarily targets online consumer devices such as IP cameras and home routers. The Mirai botnet was first found in August 2016 by MalwareMustDie, a white hat malware research group, and has been used in some of the largest and most disruptive distributed denial of service (DDoS) attacks, including an attack on 20 September 2016 on computer security journalist Brian Krebs' website, an attack on French web host OVH, and the October 2016 Dyn cyberattack. According to a chat log between Anna-senpai and Robert Coelho, Mirai was named after the 2011 TV anime series Mirai Nikki. The software was initially used by the creators to DDoS Minecraft servers and companies offering DDoS protection to Minecraft servers, with the authors using Mirai to operate a protection racket. The source code for Mirai was subsequently published on Hack Forums as open-source. Since the source code was published, the techniques have been adapted in other malware projects."),

    ("Network_Security_Services", "type", "Software"),
    ("Network_Security_Services", "label", "Network Security Services"),
    ("Network_Security_Services", "abstract", "Network Security Services (NSS) is a collection of cryptographic computer libraries designed to support cross-platform development of security-enabled client and server applications with optional support for hardware TLS/SSL acceleration on the server side and hardware smart cards on the client side. NSS provides a complete open-source implementation of cryptographic libraries supporting Transport Layer Security (TLS) / Secure Sockets Layer (SSL) and S/MIME. NSS releases prior to version 3.14 are tri-licensed under the Mozilla Public License 1.1, the GNU General Public License, and the GNU Lesser General Public License. Since release 3.14, NSS releases are licensed under GPL-compatible Mozilla Public License 2.0."),

    ("MS_Antivirus_(malware)", "type", "Software"),
    ("MS_Antivirus_(malware)", "label", "MS Antivirus (malware)"),
    ("MS_Antivirus_(malware)", "abstract", "MS Antivirus (also known as Spyware Protect 2009 and Antivirus XP 2008/Antivirus2009/SecurityTool/etc) is a scareware rogue anti-virus which purports to remove virus infections found on a computer running Microsoft Windows. It attempts to scam the user into purchasing a 'full version' of the software. The company and the individuals behind Bakasoftware operated under other different 'company' names, including Innovagest2000, Innovative Marketing Ukraine, Pandora Software, LocusSoftware, etc."),
    ("MS_Antivirus_(malware)","developer" ,"Bakasoftware"),

    ("CyberCIEGE", "type", "Software"),
    ("CyberCIEGE", "label", "CyberCIEGE"),
    ("CyberCIEGE", "abstract", "CyberCIEGE is a serious game designed to teach network security concepts. Its development was sponsored by the U.S. Navy, and it is used as a training tool by agencies of the U.S. government, universities and community colleges. CyberCIEGE covers a broad range of cybersecurity topics. Players purchase and configure computers and network devices to keep demanding users happy (e.g., by providing Internet access) all while protecting assets from a variety of attacks. The game includes a number of different scenarios, some of which focus on basic training and awareness, others on more advanced network security concepts."),

    ("CyberExtension", "type", "Software"),
    ("CyberExtension", "label", "CyberExtension"),
    ("CyberExtension", "abstract", "CyberExtension is a managed virtual learning environment built by Right Reason Technologies (RRT) that is designed to be used as an online extension of a school district in the USA. The system is currently used for homebound students, credit recovery and supplemental coursework, and has been successfully deployed to help students with 'school phobia' and students undergoing medical treatment."),

    ("CyberGladiators", "type", "Software"),
    ("CyberGladiators", "label", "CyberGladiators"),
    ("CyberGladiators", "abstract", "CyberGladiators is a 1996 fighting game developed by K.A.A. (label of Dynamix) and published by Sierra On-Line. The game is divided of fights between members of the Quaaflax Alliance and members of the terrorist organization Gy Djin. All characters have been transformed by a cosmic storm into CyberGladiators."),

    ("CyberLink_MediaShow", "type", "Software"),
    ("CyberLink_MediaShow", "label", "CyberLink MediaShow"),
    ("CyberLink_MediaShow", "abstract", "CyberLink MediaShow is a software application for organizing, editing and sharing photos and videos, published by CyberLink Corporation. MediaShow allows users to import video and photo files from digital cameras, phones, and camcorders. Its main competitors are Arcsoft MediaImpression, Adobe Bridge, Roxio Creator and Google Picasa."),

    ("CyberQuery", "type", "Software"),
    ("CyberQuery", "label", "CyberQuery"),
    ("CyberQuery", "abstract", "Cyberquery is a software product of Cyberscience Corporation Inc. Originally developed for data handling and analysis on Data General AOS and AOS/VS minicomputers, then the available platforms for Cyberquery were extended to all major UNIX platforms, OpenVMS, and Microsoft Windows."),

    ("CyberStrike", "type", "Software"),
    ("CyberStrike", "label", "CyberStrike"),
    ("CyberStrike", "abstract", "CyberStrike is a futuristic 3D combat online game by Simutronics, involving team combat between customizable mechs, each of which is controlled by a different player. Initially exclusive to the GEnie online service, it opened in February 1993, and later that year it caused Computer Gaming World to create the new category of 'Online Game of the Year' so it could be awarded to CyberStrike. In 1994, it was offered in stores by MicroProse. In 1997, a sequel, Cyberstrike 2, was announced as a joint project between Simutronics and Sony's 989 Studios.")
]
part_indices_added = {
    "Part 1": range(0, 3),   # Scooby-Doo_and_the_Cyber_Chase_(video_game) (3 attributes)
    "Part 2": range(3, 6),   # Mirai_(malware) (3 attributes)
    "Part 3": range(6, 9),   # Network_Security_Services (3 attributes)
    "Part 4": range(9, 13),  # MS_Antivirus_(malware) (4 attributes, including developer)
    "Part 5": range(13, 16), # CyberCIEGE (3 attributes)
    "Part 6": range(16, 19), # CyberExtension (3 attributes)
    "Part 7": range(19, 22), # CyberGladiators (3 attributes)
    "Part 8": range(22, 25), # CyberLink_MediaShow (3 attributes)
    "Part 9": range(25, 28), # CyberQuery (3 attributes)
    "Part 10": range(28, 31) # CyberStrike (3 attributes)
}

part_names_added = list(part_indices_added.keys())


In [ ]:
visualize_graph_with_chains(kg, part_indices)

In [ ]:
temp = 1
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(original_answer_str)

In [ ]:
temp= 1
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.074, -0.349,  0.278, -0.24 , -0.256,  0.23 ,  0.057, -0.081,
        0.373, -0.109])
coeff_added = np.array([-0.06557593, -0.28650022,  0.25216548,  0.13783945,  0.04449362,
       -0.01817797,  0.13227846,  0.05871761, -0.20607784,  0.35022204])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 0) by introducing small perturbations in the input or graph structure -(MS_Antivirus_(malware)) → (developer) → (Bakasoftware)- and assessing the consistency of generated explanations.

In [ ]:
temp = 0
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(f"Original answer: {original_answer_str}")

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.,  0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0.])
coeff_added = np.array([-4.09187947e-06,  1.69596629e-05,  9.99908412e-01,  1.48769811e-05,
        2.83603444e-06, -1.32556869e-05,  2.66331559e-05,  1.91159129e-05,
       -1.92008939e-05,  1.31978667e-05])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 1) by introducing small perturbations in the input or graph structure -(CyberCIEGE) → (developer) → (Naval_Postgraduate_School)- and assessing the consistency of generated explanations.

In [ ]:
kg_added = [
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "type", "Software"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "label", "Scooby-Doo and the Cyber Chase (video game)"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "abstract", "Scooby-Doo and the Cyber Chase is a Scooby-Doo video game based on the Warner Brothers film Scooby-Doo and the Cyber Chase. The game was released for the PlayStation and Game Boy Advance in 2001. The PlayStation version became a 'Greatest Hits' title in 2003."),

    ("Mirai_(malware)", "type", "Software"),
    ("Mirai_(malware)", "label", "Mirai (malware)"),
    ("Mirai_(malware)", "abstract", "Mirai (from the Japanese word for 'future', 未来) is a malware that turns networked devices running Linux into remotely controlled bots that can be used as part of a botnet in large-scale network attacks. It primarily targets online consumer devices such as IP cameras and home routers. The Mirai botnet was first found in August 2016 by MalwareMustDie, a white hat malware research group, and has been used in some of the largest and most disruptive distributed denial of service (DDoS) attacks, including an attack on 20 September 2016 on computer security journalist Brian Krebs' website, an attack on French web host OVH, and the October 2016 Dyn cyberattack. According to a chat log between Anna-senpai and Robert Coelho, Mirai was named after the 2011 TV anime series Mirai Nikki. The software was initially used by the creators to DDoS Minecraft servers and companies offering DDoS protection to Minecraft servers, with the authors using Mirai to operate a protection racket. The source code for Mirai was subsequently published on Hack Forums as open-source. Since the source code was published, the techniques have been adapted in other malware projects."),

    ("Network_Security_Services", "type", "Software"),
    ("Network_Security_Services", "label", "Network Security Services"),
    ("Network_Security_Services", "abstract", "Network Security Services (NSS) is a collection of cryptographic computer libraries designed to support cross-platform development of security-enabled client and server applications with optional support for hardware TLS/SSL acceleration on the server side and hardware smart cards on the client side. NSS provides a complete open-source implementation of cryptographic libraries supporting Transport Layer Security (TLS) / Secure Sockets Layer (SSL) and S/MIME. NSS releases prior to version 3.14 are tri-licensed under the Mozilla Public License 1.1, the GNU General Public License, and the GNU Lesser General Public License. Since release 3.14, NSS releases are licensed under GPL-compatible Mozilla Public License 2.0."),

    ("MS_Antivirus_(malware)", "type", "Software"),
    ("MS_Antivirus_(malware)", "label", "MS Antivirus (malware)"),
    ("MS_Antivirus_(malware)", "abstract", "MS Antivirus (also known as Spyware Protect 2009 and Antivirus XP 2008/Antivirus2009/SecurityTool/etc) is a scareware rogue anti-virus which purports to remove virus infections found on a computer running Microsoft Windows. It attempts to scam the user into purchasing a 'full version' of the software. The company and the individuals behind Bakasoftware operated under other different 'company' names, including Innovagest2000, Innovative Marketing Ukraine, Pandora Software, LocusSoftware, etc."),

    ("CyberCIEGE", "type", "Software"),
    ("CyberCIEGE", "label", "CyberCIEGE"),
    ("CyberCIEGE", "abstract", "CyberCIEGE is a serious game designed to teach network security concepts. Its development was sponsored by the U.S. Navy, and it is used as a training tool by agencies of the U.S. government, universities and community colleges. CyberCIEGE covers a broad range of cybersecurity topics. Players purchase and configure computers and network devices to keep demanding users happy (e.g., by providing Internet access) all while protecting assets from a variety of attacks. The game includes a number of different scenarios, some of which focus on basic training and awareness, others on more advanced network security concepts."),
    ("CyberCIEGE","developer", "Naval_Postgraduate_School"),

    ("CyberExtension", "type", "Software"),
    ("CyberExtension", "label", "CyberExtension"),
    ("CyberExtension", "abstract", "CyberExtension is a managed virtual learning environment built by Right Reason Technologies (RRT) that is designed to be used as an online extension of a school district in the USA. The system is currently used for homebound students, credit recovery and supplemental coursework, and has been successfully deployed to help students with 'school phobia' and students undergoing medical treatment."),

    ("CyberGladiators", "type", "Software"),
    ("CyberGladiators", "label", "CyberGladiators"),
    ("CyberGladiators", "abstract", "CyberGladiators is a 1996 fighting game developed by K.A.A. (label of Dynamix) and published by Sierra On-Line. The game is divided of fights between members of the Quaaflax Alliance and members of the terrorist organization Gy Djin. All characters have been transformed by a cosmic storm into CyberGladiators."),

    ("CyberLink_MediaShow", "type", "Software"),
    ("CyberLink_MediaShow", "label", "CyberLink MediaShow"),
    ("CyberLink_MediaShow", "abstract", "CyberLink MediaShow is a software application for organizing, editing and sharing photos and videos, published by CyberLink Corporation. MediaShow allows users to import video and photo files from digital cameras, phones, and camcorders. Its main competitors are Arcsoft MediaImpression, Adobe Bridge, Roxio Creator and Google Picasa."),

    ("CyberQuery", "type", "Software"),
    ("CyberQuery", "label", "CyberQuery"),
    ("CyberQuery", "abstract", "Cyberquery is a software product of Cyberscience Corporation Inc. Originally developed for data handling and analysis on Data General AOS and AOS/VS minicomputers, then the available platforms for Cyberquery were extended to all major UNIX platforms, OpenVMS, and Microsoft Windows."),

    ("CyberStrike", "type", "Software"),
    ("CyberStrike", "label", "CyberStrike"),
    ("CyberStrike", "abstract", "CyberStrike is a futuristic 3D combat online game by Simutronics, involving team combat between customizable mechs, each of which is controlled by a different player. Initially exclusive to the GEnie online service, it opened in February 1993, and later that year it caused Computer Gaming World to create the new category of 'Online Game of the Year' so it could be awarded to CyberStrike. In 1994, it was offered in stores by MicroProse. In 1997, a sequel, Cyberstrike 2, was announced as a joint project between Simutronics and Sony's 989 Studios.")
]
part_indices_added = {
    "Part 1": range(0, 3),   # Scooby-Doo_and_the_Cyber_Chase_(video_game) (3 attributes)
    "Part 2": range(3, 6),   # Mirai_(malware) (3 attributes)
    "Part 3": range(6, 9),   # Network_Security_Services (3 attributes)
    "Part 4": range(9, 12),  # MS_Antivirus_(malware) (3 attributes)
    "Part 5": range(12, 16), # CyberCIEGE (4 attributes, including developer)
    "Part 6": range(16, 19), # CyberExtension (3 attributes)
    "Part 7": range(19, 22), # CyberGladiators (3 attributes)
    "Part 8": range(22, 25), # CyberLink_MediaShow (3 attributes)
    "Part 9": range(25, 28), # CyberQuery (3 attributes)
    "Part 10": range(28, 31) # CyberStrike (3 attributes)
}

part_names_added = list(part_indices_added.keys())


In [ ]:
visualize_graph_with_chains(kg, part_indices)

In [ ]:
temp = 1
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(f"Original answer: {original_answer_str}")

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.074, -0.349,  0.278, -0.24 , -0.256,  0.23 ,  0.057, -0.081,
        0.373, -0.109])
coeff_added = np.array([ 0.14319166, -0.02258273, -0.25452967,  0.37758658, -0.29612795,
        0.08548098,  0.06857783,  0.16150876,  0.06706322,  0.21985503])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 0) by introducing small perturbations in the input or graph structure -(CyberCIEGE) → (developer) → (Naval_Postgraduate_School)- and assessing the consistency of generated explanations.

In [ ]:
temp = 0
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(f"Original answer: {original_answer_str}")

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.,  0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0.])
coeff_added =np.array([-8.01450204e-06, -5.46858460e-05,  9.99903293e-01, -2.17888417e-05,
        1.41712491e-05,  2.24151886e-05,  5.44893480e-05, -7.80482253e-06,
        1.18546742e-05,  2.90112684e-05])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 1) by introducing small perturbations in the input or graph structure -(CyberExtension) → (developer) → (Right_Reason_Technologies)- and assessing the consistency of generated explanations.

In [ ]:
kg_added = [
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "type", "Software"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "label", "Scooby-Doo and the Cyber Chase (video game)"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "abstract", "Scooby-Doo and the Cyber Chase is a Scooby-Doo video game based on the Warner Brothers film Scooby-Doo and the Cyber Chase. The game was released for the PlayStation and Game Boy Advance in 2001. The PlayStation version became a 'Greatest Hits' title in 2003."),

    ("Mirai_(malware)", "type", "Software"),
    ("Mirai_(malware)", "label", "Mirai (malware)"),
    ("Mirai_(malware)", "abstract", "Mirai (from the Japanese word for 'future', 未来) is a malware that turns networked devices running Linux into remotely controlled bots that can be used as part of a botnet in large-scale network attacks. It primarily targets online consumer devices such as IP cameras and home routers. The Mirai botnet was first found in August 2016 by MalwareMustDie, a white hat malware research group, and has been used in some of the largest and most disruptive distributed denial of service (DDoS) attacks, including an attack on 20 September 2016 on computer security journalist Brian Krebs' website, an attack on French web host OVH, and the October 2016 Dyn cyberattack. According to a chat log between Anna-senpai and Robert Coelho, Mirai was named after the 2011 TV anime series Mirai Nikki. The software was initially used by the creators to DDoS Minecraft servers and companies offering DDoS protection to Minecraft servers, with the authors using Mirai to operate a protection racket. The source code for Mirai was subsequently published on Hack Forums as open-source. Since the source code was published, the techniques have been adapted in other malware projects."),

    ("Network_Security_Services", "type", "Software"),
    ("Network_Security_Services", "label", "Network Security Services"),
    ("Network_Security_Services", "abstract", "Network Security Services (NSS) is a collection of cryptographic computer libraries designed to support cross-platform development of security-enabled client and server applications with optional support for hardware TLS/SSL acceleration on the server side and hardware smart cards on the client side. NSS provides a complete open-source implementation of cryptographic libraries supporting Transport Layer Security (TLS) / Secure Sockets Layer (SSL) and S/MIME. NSS releases prior to version 3.14 are tri-licensed under the Mozilla Public License 1.1, the GNU General Public License, and the GNU Lesser General Public License. Since release 3.14, NSS releases are licensed under GPL-compatible Mozilla Public License 2.0."),

    ("MS_Antivirus_(malware)", "type", "Software"),
    ("MS_Antivirus_(malware)", "label", "MS Antivirus (malware)"),
    ("MS_Antivirus_(malware)", "abstract", "MS Antivirus (also known as Spyware Protect 2009 and Antivirus XP 2008/Antivirus2009/SecurityTool/etc) is a scareware rogue anti-virus which purports to remove virus infections found on a computer running Microsoft Windows. It attempts to scam the user into purchasing a 'full version' of the software. The company and the individuals behind Bakasoftware operated under other different 'company' names, including Innovagest2000, Innovative Marketing Ukraine, Pandora Software, LocusSoftware, etc."),

    ("CyberCIEGE", "type", "Software"),
    ("CyberCIEGE", "label", "CyberCIEGE"),
    ("CyberCIEGE", "abstract", "CyberCIEGE is a serious game designed to teach network security concepts. Its development was sponsored by the U.S. Navy, and it is used as a training tool by agencies of the U.S. government, universities and community colleges. CyberCIEGE covers a broad range of cybersecurity topics. Players purchase and configure computers and network devices to keep demanding users happy (e.g., by providing Internet access) all while protecting assets from a variety of attacks. The game includes a number of different scenarios, some of which focus on basic training and awareness, others on more advanced network security concepts."),

    ("CyberExtension", "type", "Software"),
    ("CyberExtension", "label", "CyberExtension"),
    ("CyberExtension", "abstract", "CyberExtension is a managed virtual learning environment built by Right Reason Technologies (RRT) that is designed to be used as an online extension of a school district in the USA. The system is currently used for homebound students, credit recovery and supplemental coursework, and has been successfully deployed to help students with 'school phobia' and students undergoing medical treatment."),
    ("CyberExtension","developer","Right_Reason_Technologies"),

    ("CyberGladiators", "type", "Software"),
    ("CyberGladiators", "label", "CyberGladiators"),
    ("CyberGladiators", "abstract", "CyberGladiators is a 1996 fighting game developed by K.A.A. (label of Dynamix) and published by Sierra On-Line. The game is divided of fights between members of the Quaaflax Alliance and members of the terrorist organization Gy Djin. All characters have been transformed by a cosmic storm into CyberGladiators."),

    ("CyberLink_MediaShow", "type", "Software"),
    ("CyberLink_MediaShow", "label", "CyberLink MediaShow"),
    ("CyberLink_MediaShow", "abstract", "CyberLink MediaShow is a software application for organizing, editing and sharing photos and videos, published by CyberLink Corporation. MediaShow allows users to import video and photo files from digital cameras, phones, and camcorders. Its main competitors are Arcsoft MediaImpression, Adobe Bridge, Roxio Creator and Google Picasa."),

    ("CyberQuery", "type", "Software"),
    ("CyberQuery", "label", "CyberQuery"),
    ("CyberQuery", "abstract", "Cyberquery is a software product of Cyberscience Corporation Inc. Originally developed for data handling and analysis on Data General AOS and AOS/VS minicomputers, then the available platforms for Cyberquery were extended to all major UNIX platforms, OpenVMS, and Microsoft Windows."),

    ("CyberStrike", "type", "Software"),
    ("CyberStrike", "label", "CyberStrike"),
    ("CyberStrike", "abstract", "CyberStrike is a futuristic 3D combat online game by Simutronics, involving team combat between customizable mechs, each of which is controlled by a different player. Initially exclusive to the GEnie online service, it opened in February 1993, and later that year it caused Computer Gaming World to create the new category of 'Online Game of the Year' so it could be awarded to CyberStrike. In 1994, it was offered in stores by MicroProse. In 1997, a sequel, Cyberstrike 2, was announced as a joint project between Simutronics and Sony's 989 Studios.")
]
part_indices_added = {
    "Part 1": range(0, 3),   # Scooby-Doo_and_the_Cyber_Chase_(video_game) (3 attributes)
    "Part 2": range(3, 6),   # Mirai_(malware) (3 attributes)
    "Part 3": range(6, 9),   # Network_Security_Services (3 attributes)
    "Part 4": range(9, 12),  # MS_Antivirus_(malware) (3 attributes)
    "Part 5": range(12, 15), # CyberCIEGE (3 attributes)
    "Part 6": range(15, 19), # CyberExtension (4 attributes, including developer)
    "Part 7": range(19, 22), # CyberGladiators (3 attributes)
    "Part 8": range(22, 25), # CyberLink_MediaShow (3 attributes)
    "Part 9": range(25, 28), # CyberQuery (3 attributes)
    "Part 10": range(28, 31) # CyberStrike (3 attributes)
}

part_names_added = list(part_indices_added.keys())



In [ ]:
visualize_graph_with_chains(kg, part_indices)

In [ ]:
temp = 1
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(original_answer_str)

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.074, -0.349,  0.278, -0.24 , -0.256,  0.23 ,  0.057, -0.081,
        0.373, -0.109])
coeff_added = np.array([ 0.10678415,  0.22178868,  0.20185825, -0.23858348, -0.11948368,
        0.08284135,  0.12474184,  0.01595862,  0.05339906, -0.259592  ])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 0) by introducing small perturbations in the input or graph structure -(CyberExtension) → (developer) → (Right_Reason_Technologies)- and assessing the consistency of generated explanations.

In [ ]:
temp = 0
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(f"Original answer: {original_answer_str}")

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.,  0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0.])
coeff_added = np.array([-4.08289921e-06,  2.49326864e-05,  9.99956037e-01,  2.40410005e-05,
       -1.41619934e-05, -4.39287151e-05, -5.15968390e-06,  1.73034205e-05,
       -1.35211510e-05,  1.14660929e-06])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 1) by introducing small perturbations in the input or graph structure -(CyberGladiators) → (publisher) → (Sierra On-Line)- and assessing the consistency of generated explanations.

In [ ]:
kg_added = [
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "type", "Software"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "label", "Scooby-Doo and the Cyber Chase (video game)"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "abstract", "Scooby-Doo and the Cyber Chase is a Scooby-Doo video game based on the Warner Brothers film Scooby-Doo and the Cyber Chase. The game was released for the PlayStation and Game Boy Advance in 2001. The PlayStation version became a 'Greatest Hits' title in 2003."),

    ("Mirai_(malware)", "type", "Software"),
    ("Mirai_(malware)", "label", "Mirai (malware)"),
    ("Mirai_(malware)", "abstract", "Mirai (from the Japanese word for 'future', 未来) is a malware that turns networked devices running Linux into remotely controlled bots that can be used as part of a botnet in large-scale network attacks. It primarily targets online consumer devices such as IP cameras and home routers. The Mirai botnet was first found in August 2016 by MalwareMustDie, a white hat malware research group, and has been used in some of the largest and most disruptive distributed denial of service (DDoS) attacks, including an attack on 20 September 2016 on computer security journalist Brian Krebs' website, an attack on French web host OVH, and the October 2016 Dyn cyberattack. According to a chat log between Anna-senpai and Robert Coelho, Mirai was named after the 2011 TV anime series Mirai Nikki. The software was initially used by the creators to DDoS Minecraft servers and companies offering DDoS protection to Minecraft servers, with the authors using Mirai to operate a protection racket. The source code for Mirai was subsequently published on Hack Forums as open-source. Since the source code was published, the techniques have been adapted in other malware projects."),

    ("Network_Security_Services", "type", "Software"),
    ("Network_Security_Services", "label", "Network Security Services"),
    ("Network_Security_Services", "abstract", "Network Security Services (NSS) is a collection of cryptographic computer libraries designed to support cross-platform development of security-enabled client and server applications with optional support for hardware TLS/SSL acceleration on the server side and hardware smart cards on the client side. NSS provides a complete open-source implementation of cryptographic libraries supporting Transport Layer Security (TLS) / Secure Sockets Layer (SSL) and S/MIME. NSS releases prior to version 3.14 are tri-licensed under the Mozilla Public License 1.1, the GNU General Public License, and the GNU Lesser General Public License. Since release 3.14, NSS releases are licensed under GPL-compatible Mozilla Public License 2.0."),

    ("MS_Antivirus_(malware)", "type", "Software"),
    ("MS_Antivirus_(malware)", "label", "MS Antivirus (malware)"),
    ("MS_Antivirus_(malware)", "abstract", "MS Antivirus (also known as Spyware Protect 2009 and Antivirus XP 2008/Antivirus2009/SecurityTool/etc) is a scareware rogue anti-virus which purports to remove virus infections found on a computer running Microsoft Windows. It attempts to scam the user into purchasing a 'full version' of the software. The company and the individuals behind Bakasoftware operated under other different 'company' names, including Innovagest2000, Innovative Marketing Ukraine, Pandora Software, LocusSoftware, etc."),

    ("CyberCIEGE", "type", "Software"),
    ("CyberCIEGE", "label", "CyberCIEGE"),
    ("CyberCIEGE", "abstract", "CyberCIEGE is a serious game designed to teach network security concepts. Its development was sponsored by the U.S. Navy, and it is used as a training tool by agencies of the U.S. government, universities and community colleges. CyberCIEGE covers a broad range of cybersecurity topics. Players purchase and configure computers and network devices to keep demanding users happy (e.g., by providing Internet access) all while protecting assets from a variety of attacks. The game includes a number of different scenarios, some of which focus on basic training and awareness, others on more advanced network security concepts."),

    ("CyberExtension", "type", "Software"),
    ("CyberExtension", "label", "CyberExtension"),
    ("CyberExtension", "abstract", "CyberExtension is a managed virtual learning environment built by Right Reason Technologies (RRT) that is designed to be used as an online extension of a school district in the USA. The system is currently used for homebound students, credit recovery and supplemental coursework, and has been successfully deployed to help students with 'school phobia' and students undergoing medical treatment."),

    ("CyberGladiators", "type", "Software"),
    ("CyberGladiators", "label", "CyberGladiators"),
    ("CyberGladiators", "abstract", "CyberGladiators is a 1996 fighting game developed by K.A.A. (label of Dynamix) and published by Sierra On-Line. The game is divided of fights between members of the Quaaflax Alliance and members of the terrorist organization Gy Djin. All characters have been transformed by a cosmic storm into CyberGladiators."),
    ("CyberGladiators","publisher","Sierra On-Line"),

    ("CyberLink_MediaShow", "type", "Software"),
    ("CyberLink_MediaShow", "label", "CyberLink MediaShow"),
    ("CyberLink_MediaShow", "abstract", "CyberLink MediaShow is a software application for organizing, editing and sharing photos and videos, published by CyberLink Corporation. MediaShow allows users to import video and photo files from digital cameras, phones, and camcorders. Its main competitors are Arcsoft MediaImpression, Adobe Bridge, Roxio Creator and Google Picasa."),

    ("CyberQuery", "type", "Software"),
    ("CyberQuery", "label", "CyberQuery"),
    ("CyberQuery", "abstract", "Cyberquery is a software product of Cyberscience Corporation Inc. Originally developed for data handling and analysis on Data General AOS and AOS/VS minicomputers, then the available platforms for Cyberquery were extended to all major UNIX platforms, OpenVMS, and Microsoft Windows."),

    ("CyberStrike", "type", "Software"),
    ("CyberStrike", "label", "CyberStrike"),
    ("CyberStrike", "abstract", "CyberStrike is a futuristic 3D combat online game by Simutronics, involving team combat between customizable mechs, each of which is controlled by a different player. Initially exclusive to the GEnie online service, it opened in February 1993, and later that year it caused Computer Gaming World to create the new category of 'Online Game of the Year' so it could be awarded to CyberStrike. In 1994, it was offered in stores by MicroProse. In 1997, a sequel, Cyberstrike 2, was announced as a joint project between Simutronics and Sony's 989 Studios.")
]
part_indices_added = {
    "Part 1": range(0, 3),   # Scooby-Doo_and_the_Cyber_Chase_(video_game) (3 attributes)
    "Part 2": range(3, 6),   # Mirai_(malware) (3 attributes)
    "Part 3": range(6, 9),   # Network_Security_Services (3 attributes)
    "Part 4": range(9, 12),  # MS_Antivirus_(malware) (3 attributes)
    "Part 5": range(12, 15), # CyberCIEGE (3 attributes)
    "Part 6": range(15, 18), # CyberExtension (3 attributes)
    "Part 7": range(18, 22), # CyberGladiators (4 attributes, including publisher)
    "Part 8": range(22, 25), # CyberLink_MediaShow (3 attributes)
    "Part 9": range(25, 28), # CyberQuery (3 attributes)
    "Part 10": range(28, 31) # CyberStrike (3 attributes)
}

part_names_added = list(part_indices_added.keys())


In [ ]:
visualize_graph_with_chains(kg, part_indices)

In [ ]:
temp = 1
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(f"Original answer: {original_answer_str}")

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.074, -0.349,  0.278, -0.24 , -0.256,  0.23 ,  0.057, -0.081,
        0.373, -0.109])
coeff_added = np.array([-0.03107438, -0.19800958,  0.08834081,  0.04978688, -0.09329162,
       -0.00692721,  0.1763618 ,  0.32043549, -0.36494622,  0.18752337])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 0) by introducing small perturbations in the input or graph structure -(CyberGladiators) → (publisher) → (Sierra On-Line)- and assessing the consistency of generated explanations.

In [ ]:
temp = 0
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(f"Original answer: {original_answer_str}")

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.,  0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0.])
coeff_added =np.array([ 4.55819005e-06,  3.28670385e-06,  9.99947363e-01,  6.37788091e-06,
        1.47029938e-05,  2.58533062e-06,  2.19099366e-06, -1.01250801e-06,
        2.85055309e-06, -2.69129808e-05])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 1) by introducing small perturbations in the input or graph structure -(CyberLink_MediaShow) → (feature) → (Photo_Editing, Video_Editing)- and assessing the consistency of generated explanations.

In [ ]:
kg_added = [
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "type", "Software"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "label", "Scooby-Doo and the Cyber Chase (video game)"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "abstract", "Scooby-Doo and the Cyber Chase is a Scooby-Doo video game based on the Warner Brothers film Scooby-Doo and the Cyber Chase. The game was released for the PlayStation and Game Boy Advance in 2001. The PlayStation version became a 'Greatest Hits' title in 2003."),

    ("Mirai_(malware)", "type", "Software"),
    ("Mirai_(malware)", "label", "Mirai (malware)"),
    ("Mirai_(malware)", "abstract", "Mirai (from the Japanese word for 'future', 未来) is a malware that turns networked devices running Linux into remotely controlled bots that can be used as part of a botnet in large-scale network attacks. It primarily targets online consumer devices such as IP cameras and home routers. The Mirai botnet was first found in August 2016 by MalwareMustDie, a white hat malware research group, and has been used in some of the largest and most disruptive distributed denial of service (DDoS) attacks, including an attack on 20 September 2016 on computer security journalist Brian Krebs' website, an attack on French web host OVH, and the October 2016 Dyn cyberattack. According to a chat log between Anna-senpai and Robert Coelho, Mirai was named after the 2011 TV anime series Mirai Nikki. The software was initially used by the creators to DDoS Minecraft servers and companies offering DDoS protection to Minecraft servers, with the authors using Mirai to operate a protection racket. The source code for Mirai was subsequently published on Hack Forums as open-source. Since the source code was published, the techniques have been adapted in other malware projects."),

    ("Network_Security_Services", "type", "Software"),
    ("Network_Security_Services", "label", "Network Security Services"),
    ("Network_Security_Services", "abstract", "Network Security Services (NSS) is a collection of cryptographic computer libraries designed to support cross-platform development of security-enabled client and server applications with optional support for hardware TLS/SSL acceleration on the server side and hardware smart cards on the client side. NSS provides a complete open-source implementation of cryptographic libraries supporting Transport Layer Security (TLS) / Secure Sockets Layer (SSL) and S/MIME. NSS releases prior to version 3.14 are tri-licensed under the Mozilla Public License 1.1, the GNU General Public License, and the GNU Lesser General Public License. Since release 3.14, NSS releases are licensed under GPL-compatible Mozilla Public License 2.0."),

    ("MS_Antivirus_(malware)", "type", "Software"),
    ("MS_Antivirus_(malware)", "label", "MS Antivirus (malware)"),
    ("MS_Antivirus_(malware)", "abstract", "MS Antivirus (also known as Spyware Protect 2009 and Antivirus XP 2008/Antivirus2009/SecurityTool/etc) is a scareware rogue anti-virus which purports to remove virus infections found on a computer running Microsoft Windows. It attempts to scam the user into purchasing a 'full version' of the software. The company and the individuals behind Bakasoftware operated under other different 'company' names, including Innovagest2000, Innovative Marketing Ukraine, Pandora Software, LocusSoftware, etc."),

    ("CyberCIEGE", "type", "Software"),
    ("CyberCIEGE", "label", "CyberCIEGE"),
    ("CyberCIEGE", "abstract", "CyberCIEGE is a serious game designed to teach network security concepts. Its development was sponsored by the U.S. Navy, and it is used as a training tool by agencies of the U.S. government, universities and community colleges. CyberCIEGE covers a broad range of cybersecurity topics. Players purchase and configure computers and network devices to keep demanding users happy (e.g., by providing Internet access) all while protecting assets from a variety of attacks. The game includes a number of different scenarios, some of which focus on basic training and awareness, others on more advanced network security concepts."),

    ("CyberExtension", "type", "Software"),
    ("CyberExtension", "label", "CyberExtension"),
    ("CyberExtension", "abstract", "CyberExtension is a managed virtual learning environment built by Right Reason Technologies (RRT) that is designed to be used as an online extension of a school district in the USA. The system is currently used for homebound students, credit recovery and supplemental coursework, and has been successfully deployed to help students with 'school phobia' and students undergoing medical treatment."),

    ("CyberGladiators", "type", "Software"),
    ("CyberGladiators", "label", "CyberGladiators"),
    ("CyberGladiators", "abstract", "CyberGladiators is a 1996 fighting game developed by K.A.A. (label of Dynamix) and published by Sierra On-Line. The game is divided of fights between members of the Quaaflax Alliance and members of the terrorist organization Gy Djin. All characters have been transformed by a cosmic storm into CyberGladiators."),

    ("CyberLink_MediaShow", "type", "Software"),
    ("CyberLink_MediaShow", "label", "CyberLink MediaShow"),
    ("CyberLink_MediaShow", "abstract", "CyberLink MediaShow is a software application for organizing, editing and sharing photos and videos, published by CyberLink Corporation. MediaShow allows users to import video and photo files from digital cameras, phones, and camcorders. Its main competitors are Arcsoft MediaImpression, Adobe Bridge, Roxio Creator and Google Picasa."),
    ("CyberLink_MediaShow","feature","Photo_Editing, Video_Editing"),

    ("CyberQuery", "type", "Software"),
    ("CyberQuery", "label", "CyberQuery"),
    ("CyberQuery", "abstract", "Cyberquery is a software product of Cyberscience Corporation Inc. Originally developed for data handling and analysis on Data General AOS and AOS/VS minicomputers, then the available platforms for Cyberquery were extended to all major UNIX platforms, OpenVMS, and Microsoft Windows."),

    ("CyberStrike", "type", "Software"),
    ("CyberStrike", "label", "CyberStrike"),
    ("CyberStrike", "abstract", "CyberStrike is a futuristic 3D combat online game by Simutronics, involving team combat between customizable mechs, each of which is controlled by a different player. Initially exclusive to the GEnie online service, it opened in February 1993, and later that year it caused Computer Gaming World to create the new category of 'Online Game of the Year' so it could be awarded to CyberStrike. In 1994, it was offered in stores by MicroProse. In 1997, a sequel, Cyberstrike 2, was announced as a joint project between Simutronics and Sony's 989 Studios.")
]
part_indices_added = {
    "Part 1": range(0, 3),   # Scooby-Doo_and_the_Cyber_Chase_(video_game) (3 attributes)
    "Part 2": range(3, 6),   # Mirai_(malware) (3 attributes)
    "Part 3": range(6, 9),   # Network_Security_Services (3 attributes)
    "Part 4": range(9, 12),  # MS_Antivirus_(malware) (3 attributes)
    "Part 5": range(12, 15), # CyberCIEGE (3 attributes)
    "Part 6": range(15, 18), # CyberExtension (3 attributes)
    "Part 7": range(18, 21), # CyberGladiators (3 attributes)
    "Part 8": range(21, 25), # CyberLink_MediaShow (4 attributes, including feature)
    "Part 9": range(25, 28), # CyberQuery (3 attributes)
    "Part 10": range(28, 31) # CyberStrike (3 attributes)
}

part_names_added = list(part_indices_added.keys())


In [ ]:
visualize_graph_with_chains(kg, part_indices)

In [ ]:
temp = 1
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(original_answer_str)

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.074, -0.349,  0.278, -0.24 , -0.256,  0.23 ,  0.057, -0.081,
        0.373, -0.109])
coeff_added = np.array([ 0.36872479,  0.1481867 , -0.05623355, -0.21282253,  0.18630449,
        0.11018158,  0.01199742, -0.21390011,  0.42020456,  0.29721487])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 0) by introducing small perturbations in the input or graph structure -(CyberLink_MediaShow) → (feature) → (Photo_Editing, Video_Editing)- and assessing the consistency of generated explanations.

In [ ]:
temp = 0
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(f"Original answer: {original_answer_str}")

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.,  0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0.])
coeff_added = np.array([-1.89823548e-06,  2.43429293e-06,  9.99923737e-01,  1.98800576e-05,
       -4.19541694e-06,  9.27024609e-06, -2.49580052e-06, -8.05556684e-07,
       -5.39495108e-06,  7.27228729e-06])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 1) by introducing small perturbations in the input or graph structure -(CyberQuery) → (platform) → (Windows, Unix, OpenVMS)- and assessing the consistency of generated explanations.

In [ ]:
kg_added = [
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "type", "Software"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "label", "Scooby-Doo and the Cyber Chase (video game)"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "abstract", "Scooby-Doo and the Cyber Chase is a Scooby-Doo video game based on the Warner Brothers film Scooby-Doo and the Cyber Chase. The game was released for the PlayStation and Game Boy Advance in 2001. The PlayStation version became a 'Greatest Hits' title in 2003."),

    ("Mirai_(malware)", "type", "Software"),
    ("Mirai_(malware)", "label", "Mirai (malware)"),
    ("Mirai_(malware)", "abstract", "Mirai (from the Japanese word for 'future', 未来) is a malware that turns networked devices running Linux into remotely controlled bots that can be used as part of a botnet in large-scale network attacks. It primarily targets online consumer devices such as IP cameras and home routers. The Mirai botnet was first found in August 2016 by MalwareMustDie, a white hat malware research group, and has been used in some of the largest and most disruptive distributed denial of service (DDoS) attacks, including an attack on 20 September 2016 on computer security journalist Brian Krebs' website, an attack on French web host OVH, and the October 2016 Dyn cyberattack. According to a chat log between Anna-senpai and Robert Coelho, Mirai was named after the 2011 TV anime series Mirai Nikki. The software was initially used by the creators to DDoS Minecraft servers and companies offering DDoS protection to Minecraft servers, with the authors using Mirai to operate a protection racket. The source code for Mirai was subsequently published on Hack Forums as open-source. Since the source code was published, the techniques have been adapted in other malware projects."),

    ("Network_Security_Services", "type", "Software"),
    ("Network_Security_Services", "label", "Network Security Services"),
    ("Network_Security_Services", "abstract", "Network Security Services (NSS) is a collection of cryptographic computer libraries designed to support cross-platform development of security-enabled client and server applications with optional support for hardware TLS/SSL acceleration on the server side and hardware smart cards on the client side. NSS provides a complete open-source implementation of cryptographic libraries supporting Transport Layer Security (TLS) / Secure Sockets Layer (SSL) and S/MIME. NSS releases prior to version 3.14 are tri-licensed under the Mozilla Public License 1.1, the GNU General Public License, and the GNU Lesser General Public License. Since release 3.14, NSS releases are licensed under GPL-compatible Mozilla Public License 2.0."),

    ("MS_Antivirus_(malware)", "type", "Software"),
    ("MS_Antivirus_(malware)", "label", "MS Antivirus (malware)"),
    ("MS_Antivirus_(malware)", "abstract", "MS Antivirus (also known as Spyware Protect 2009 and Antivirus XP 2008/Antivirus2009/SecurityTool/etc) is a scareware rogue anti-virus which purports to remove virus infections found on a computer running Microsoft Windows. It attempts to scam the user into purchasing a 'full version' of the software. The company and the individuals behind Bakasoftware operated under other different 'company' names, including Innovagest2000, Innovative Marketing Ukraine, Pandora Software, LocusSoftware, etc."),

    ("CyberCIEGE", "type", "Software"),
    ("CyberCIEGE", "label", "CyberCIEGE"),
    ("CyberCIEGE", "abstract", "CyberCIEGE is a serious game designed to teach network security concepts. Its development was sponsored by the U.S. Navy, and it is used as a training tool by agencies of the U.S. government, universities and community colleges. CyberCIEGE covers a broad range of cybersecurity topics. Players purchase and configure computers and network devices to keep demanding users happy (e.g., by providing Internet access) all while protecting assets from a variety of attacks. The game includes a number of different scenarios, some of which focus on basic training and awareness, others on more advanced network security concepts."),

    ("CyberExtension", "type", "Software"),
    ("CyberExtension", "label", "CyberExtension"),
    ("CyberExtension", "abstract", "CyberExtension is a managed virtual learning environment built by Right Reason Technologies (RRT) that is designed to be used as an online extension of a school district in the USA. The system is currently used for homebound students, credit recovery and supplemental coursework, and has been successfully deployed to help students with 'school phobia' and students undergoing medical treatment."),

    ("CyberGladiators", "type", "Software"),
    ("CyberGladiators", "label", "CyberGladiators"),
    ("CyberGladiators", "abstract", "CyberGladiators is a 1996 fighting game developed by K.A.A. (label of Dynamix) and published by Sierra On-Line. The game is divided of fights between members of the Quaaflax Alliance and members of the terrorist organization Gy Djin. All characters have been transformed by a cosmic storm into CyberGladiators."),

    ("CyberLink_MediaShow", "type", "Software"),
    ("CyberLink_MediaShow", "label", "CyberLink MediaShow"),
    ("CyberLink_MediaShow", "abstract", "CyberLink MediaShow is a software application for organizing, editing and sharing photos and videos, published by CyberLink Corporation. MediaShow allows users to import video and photo files from digital cameras, phones, and camcorders. Its main competitors are Arcsoft MediaImpression, Adobe Bridge, Roxio Creator and Google Picasa."),

    ("CyberQuery", "type", "Software"),
    ("CyberQuery", "label", "CyberQuery"),
    ("CyberQuery", "abstract", "Cyberquery is a software product of Cyberscience Corporation Inc. Originally developed for data handling and analysis on Data General AOS and AOS/VS minicomputers, then the available platforms for Cyberquery were extended to all major UNIX platforms, OpenVMS, and Microsoft Windows."),
    ("CyberQuery","platform","Windows, Unix, OpenVMS"),

    ("CyberStrike", "type", "Software"),
    ("CyberStrike", "label", "CyberStrike"),
    ("CyberStrike", "abstract", "CyberStrike is a futuristic 3D combat online game by Simutronics, involving team combat between customizable mechs, each of which is controlled by a different player. Initially exclusive to the GEnie online service, it opened in February 1993, and later that year it caused Computer Gaming World to create the new category of 'Online Game of the Year' so it could be awarded to CyberStrike. In 1994, it was offered in stores by MicroProse. In 1997, a sequel, Cyberstrike 2, was announced as a joint project between Simutronics and Sony's 989 Studios.")
]
part_indices_added = {
    "Part 1": range(0, 3),   # Scooby-Doo_and_the_Cyber_Chase_(video_game) (3 attributes)
    "Part 2": range(3, 6),   # Mirai_(malware) (3 attributes)
    "Part 3": range(6, 9),   # Network_Security_Services (3 attributes)
    "Part 4": range(9, 12),  # MS_Antivirus_(malware) (3 attributes)
    "Part 5": range(12, 15), # CyberCIEGE (3 attributes)
    "Part 6": range(15, 18), # CyberExtension (3 attributes)
    "Part 7": range(18, 21), # CyberGladiators (3 attributes)
    "Part 8": range(21, 24), # CyberLink_MediaShow (3 attributes)
    "Part 9": range(24, 28), # CyberQuery (4 attributes, including platform)
    "Part 10": range(28, 31) # CyberStrike (3 attributes)
}

part_names_added = list(part_indices_added.keys())



In [ ]:
visualize_graph_with_chains(kg, part_indices)

In [ ]:
temp = 1
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(original_answer_str)

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.074, -0.349,  0.278, -0.24 , -0.256,  0.23 ,  0.057, -0.081,
        0.373, -0.109])
coeff_added = np.array([ 1.30622777,  0.19885919,  0.22188975, -0.66733012, -0.34512841,
        0.10683203,  0.53300889, -0.65396778,  0.10628119, -0.01095575])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 0) by introducing small perturbations in the input or graph structure -(CyberQuery) → (platform) → (Windows, Unix, OpenVMS)- and assessing the consistency of generated explanations.

In [ ]:
temp = 0
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(f"Original answer: {original_answer_str}")

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.,  0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0.])
coeff_added = np.array([ 5.65870483e-06, -1.81987481e-06,  9.99931971e-01,  7.32212663e-06,
       -7.21300851e-06, -3.81859816e-06, -4.53506413e-06,  1.47576528e-05,
        1.65984939e-05, -4.42069656e-06])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 1) by introducing small perturbations in the input or graph structure -(CyberStrike) → (developer) → (Simutronics)- and assessing the consistency of generated explanations.

In [ ]:
kg_added = [
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "type", "Software"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "label", "Scooby-Doo and the Cyber Chase (video game)"),
    ("Scooby-Doo_and_the_Cyber_Chase_(video_game)", "abstract", "Scooby-Doo and the Cyber Chase is a Scooby-Doo video game based on the Warner Brothers film Scooby-Doo and the Cyber Chase. The game was released for the PlayStation and Game Boy Advance in 2001. The PlayStation version became a 'Greatest Hits' title in 2003."),

    ("Mirai_(malware)", "type", "Software"),
    ("Mirai_(malware)", "label", "Mirai (malware)"),
    ("Mirai_(malware)", "abstract", "Mirai (from the Japanese word for 'future', 未来) is a malware that turns networked devices running Linux into remotely controlled bots that can be used as part of a botnet in large-scale network attacks. It primarily targets online consumer devices such as IP cameras and home routers. The Mirai botnet was first found in August 2016 by MalwareMustDie, a white hat malware research group, and has been used in some of the largest and most disruptive distributed denial of service (DDoS) attacks, including an attack on 20 September 2016 on computer security journalist Brian Krebs' website, an attack on French web host OVH, and the October 2016 Dyn cyberattack. According to a chat log between Anna-senpai and Robert Coelho, Mirai was named after the 2011 TV anime series Mirai Nikki. The software was initially used by the creators to DDoS Minecraft servers and companies offering DDoS protection to Minecraft servers, with the authors using Mirai to operate a protection racket. The source code for Mirai was subsequently published on Hack Forums as open-source. Since the source code was published, the techniques have been adapted in other malware projects."),

    ("Network_Security_Services", "type", "Software"),
    ("Network_Security_Services", "label", "Network Security Services"),
    ("Network_Security_Services", "abstract", "Network Security Services (NSS) is a collection of cryptographic computer libraries designed to support cross-platform development of security-enabled client and server applications with optional support for hardware TLS/SSL acceleration on the server side and hardware smart cards on the client side. NSS provides a complete open-source implementation of cryptographic libraries supporting Transport Layer Security (TLS) / Secure Sockets Layer (SSL) and S/MIME. NSS releases prior to version 3.14 are tri-licensed under the Mozilla Public License 1.1, the GNU General Public License, and the GNU Lesser General Public License. Since release 3.14, NSS releases are licensed under GPL-compatible Mozilla Public License 2.0."),

    ("MS_Antivirus_(malware)", "type", "Software"),
    ("MS_Antivirus_(malware)", "label", "MS Antivirus (malware)"),
    ("MS_Antivirus_(malware)", "abstract", "MS Antivirus (also known as Spyware Protect 2009 and Antivirus XP 2008/Antivirus2009/SecurityTool/etc) is a scareware rogue anti-virus which purports to remove virus infections found on a computer running Microsoft Windows. It attempts to scam the user into purchasing a 'full version' of the software. The company and the individuals behind Bakasoftware operated under other different 'company' names, including Innovagest2000, Innovative Marketing Ukraine, Pandora Software, LocusSoftware, etc."),

    ("CyberCIEGE", "type", "Software"),
    ("CyberCIEGE", "label", "CyberCIEGE"),
    ("CyberCIEGE", "abstract", "CyberCIEGE is a serious game designed to teach network security concepts. Its development was sponsored by the U.S. Navy, and it is used as a training tool by agencies of the U.S. government, universities and community colleges. CyberCIEGE covers a broad range of cybersecurity topics. Players purchase and configure computers and network devices to keep demanding users happy (e.g., by providing Internet access) all while protecting assets from a variety of attacks. The game includes a number of different scenarios, some of which focus on basic training and awareness, others on more advanced network security concepts."),

    ("CyberExtension", "type", "Software"),
    ("CyberExtension", "label", "CyberExtension"),
    ("CyberExtension", "abstract", "CyberExtension is a managed virtual learning environment built by Right Reason Technologies (RRT) that is designed to be used as an online extension of a school district in the USA. The system is currently used for homebound students, credit recovery and supplemental coursework, and has been successfully deployed to help students with 'school phobia' and students undergoing medical treatment."),

    ("CyberGladiators", "type", "Software"),
    ("CyberGladiators", "label", "CyberGladiators"),
    ("CyberGladiators", "abstract", "CyberGladiators is a 1996 fighting game developed by K.A.A. (label of Dynamix) and published by Sierra On-Line. The game is divided of fights between members of the Quaaflax Alliance and members of the terrorist organization Gy Djin. All characters have been transformed by a cosmic storm into CyberGladiators."),

    ("CyberLink_MediaShow", "type", "Software"),
    ("CyberLink_MediaShow", "label", "CyberLink MediaShow"),
    ("CyberLink_MediaShow", "abstract", "CyberLink MediaShow is a software application for organizing, editing and sharing photos and videos, published by CyberLink Corporation. MediaShow allows users to import video and photo files from digital cameras, phones, and camcorders. Its main competitors are Arcsoft MediaImpression, Adobe Bridge, Roxio Creator and Google Picasa."),

    ("CyberQuery", "type", "Software"),
    ("CyberQuery", "label", "CyberQuery"),
    ("CyberQuery", "abstract", "Cyberquery is a software product of Cyberscience Corporation Inc. Originally developed for data handling and analysis on Data General AOS and AOS/VS minicomputers, then the available platforms for Cyberquery were extended to all major UNIX platforms, OpenVMS, and Microsoft Windows."),

    ("CyberStrike", "type", "Software"),
    ("CyberStrike", "label", "CyberStrike"),
    ("CyberStrike", "abstract", "CyberStrike is a futuristic 3D combat online game by Simutronics, involving team combat between customizable mechs, each of which is controlled by a different player. Initially exclusive to the GEnie online service, it opened in February 1993, and later that year it caused Computer Gaming World to create the new category of 'Online Game of the Year' so it could be awarded to CyberStrike. In 1994, it was offered in stores by MicroProse. In 1997, a sequel, Cyberstrike 2, was announced as a joint project between Simutronics and Sony's 989 Studios."),
    ("CyberStrike","developer","Simutronics")
]
part_indices_added = {
    "Part 1": range(0, 3),   # Scooby-Doo_and_the_Cyber_Chase_(video_game) (3 attributes)
    "Part 2": range(3, 6),   # Mirai_(malware) (3 attributes)
    "Part 3": range(6, 9),   # Network_Security_Services (3 attributes)
    "Part 4": range(9, 12),  # MS_Antivirus_(malware) (3 attributes)
    "Part 5": range(12, 15), # CyberCIEGE (3 attributes)
    "Part 6": range(15, 18), # CyberExtension (3 attributes)
    "Part 7": range(18, 21), # CyberGladiators (3 attributes)
    "Part 8": range(21, 24), # CyberLink_MediaShow (3 attributes)
    "Part 9": range(24, 27), # CyberQuery (3 attributes)
    "Part 10": range(27, 31) # CyberStrike (4 attributes, including developer)
}

part_names_added = list(part_indices_added.keys())



In [ ]:
visualize_graph_with_chains(kg, part_indices)

In [ ]:
temp = 1
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(f"Original answer: {original_answer_str}")

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.074, -0.349,  0.278, -0.24 , -0.256,  0.23 ,  0.057, -0.081,
        0.373, -0.109])
coeff_added =np.array([-0.02234258, -0.01757465,  0.20638687,  0.24100065,  0.159924  ,
        0.18545658,  0.00851473,  0.05492822,  0.02198553, -0.04757603])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)

## Stability: Evaluate the explanation for "what is Network_Security_Services?" (Temp 0) by introducing small perturbations in the input or graph structure -(CyberStrike) → (developer) → (Simutronics)- and assessing the consistency of generated explanations.

In [ ]:
temp = 0
original_answer_str, original_answer_embedding = get_answer_and_embedding(question, temp, graph)
print(f"Original answer: {original_answer_str}")

In [ ]:
coeff = calculate_coefficients_print_Temerature(
    temp= temp,
    original=original,
    kg=kg_added,
    part_names=part_names_added,
    question=question,
    original_answer_embedding=original_answer_embedding,
    original_answer_str = original_answer_str,
)

In [ ]:
coeff

In [ ]:
# Define the coefficient arrays
coeff_original = np.array([ 0.,  0.,  1.,  0., -0.,  0.,  0.,  0.,  0., -0.])
coeff_added = np.array([ 1.45335520e-06, -1.79713148e-05,  9.99959183e-01, -2.87385559e-05,
       -1.12191224e-05, -2.33819231e-06,  2.33958338e-05,  2.30942002e-05,
       -2.17664885e-06, -7.10879282e-06])
# Call the function
plot_knowledge_graph_explainability_compare(
    kg_original= kg,
    coeff_original=coeff_original,
    part_indices_original=part_indices,
    kg_added=kg_added,
    coeff_added=coeff_added,
    part_indices_added=part_indices_added
)

In [ ]:
jaccard_index = calculate_jaccard_index(coeff_original, coeff_added)
print("Jaccard Index:", jaccard_index)